## Controle Quântico Ótimo - Utilização do CG para ajuste do caminho percorrido por uma partícula

Nessa etapa, é necessário avaliar, a partir da equação de Heisenberg, com o método de Runge-Kutta e com o controle MPC, como a curva de origem ajusta-se a curva de destino. Além disso, é interessantíssimo a avaliação a partir do estado estacionário e ainda verificar como ocorre o ajuste de fases.

Para analisar como o controle melhora o condicionamento da curva através da sua análise futura, esse arquivo estará analisando os ajustes no horizonte 3 ao 10.

In [1]:
# Bilbiotecas para auxílio na programação matemática
import math, sys 
import numpy as np
import sympy as sp

from scipy import sparse # Produção das diagonais das matrizes
from scipy.sparse import diags 

# Plotagem 2D e 3D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm


from os import path # Suficiente para manipulação de arquivos
    
# Para solução exata
from scipy.special import hermite
from math import factorial

%matplotlib inline
count = 0

# Para otimização dos sistemas
from scipy import optimize

import random

Abaixo se dão os requisitos básicos para a instauração do teste. Se tem formas de subtrair e somar matrizes, a manipulação das derivadas com o Runge-Kutta além da própria proposta da função objetivo.

In [2]:
# Manipulação das matrizes

def somar(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] + B[i][j]

    return C

def sub(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] - B[i][j]

    return C


#################################################################

## d (psi) / dt = -i * H * |psi>
def dpsidt(t, psi, H): # A derivada da onda em relação ao tempo não tem dependência temporal
    A = np.zeros((2,2), dtype=np.complex_)
    A = np.dot(complex(0,1),H) # i * H
    return -1*np.matmul(A,psi) # - i * H * |psi>

# Runge-Kutta de quarta ordem

def rungeKutta(onda, fatorRungeKutta, hamiltoniano, tempoFinal = 2, tempoInicial = 0):
    
    resultado = np.zeros((2,int(100*tempoFinal)), dtype=np.complex_)
    indice = 0

    for i in range(0, 2):
        
        resultado[0][indice] = onda[0][0]
        resultado[1][indice] = onda[1][0]
        
        k1 = dpsidt(tempoInicial, onda, hamiltoniano)
        k2 = dpsidt(tempoInicial + 0.5 * fatorRungeKutta, somar(onda, np.dot((0.5*fatorRungeKutta), k1)), hamiltoniano)
        k3 = dpsidt(tempoInicial + 0.5 * fatorRungeKutta, somar(onda, np.dot((0.5*fatorRungeKutta), k2)), hamiltoniano)
        k4 = dpsidt(tempoInicial + fatorRungeKutta, somar(onda, np.dot(fatorRungeKutta, k3)), hamiltoniano)
        
        ## y(i+1) = y(i) + h/6*(k1+2*k2+2*k3+k4)
        
        A = somar(np.dot(2,k3), k4)
        B = somar(np.dot(2,k2), k1)
        C = somar(A, B)
 
        onda = somar(onda,np.dot((fatorRungeKutta / 6.0),(C)))
    
        tempoInicial += fatorRungeKutta
        
        indice += 1
        
    runge = np.zeros((2,1), dtype=np.complex_)
    runge[0][0] = resultado[0][indice-1]
    runge[1][0] = resultado[1][indice-1]
    return runge

def funcaoObjetivo(x, iteracao, ondaDestino, hamiltoniano, onda, horizonteAnalisavel, fatorRungeKutta, tempoAnalise):
    
    avanco = np.zeros((2,1),dtype=np.complex_)
    
    y = 0
    
    horizonte = (horizonteAnalisavel+iteracao)/100
    
    tempo = iteracao/100
    
    tempoInicialAnalise = 0.00
    tempoFinalAnalise = 0.02
    
    controles = np.zeros((2,2), dtype=np.complex_)
    controles = [[0, x[0]], [x[0], 0]]
    
    matrizOrigem = np.zeros((2,1),dtype=np.complex_)
    matrizOrigem = [[onda[0][0]], [onda[1][0]]] 

    matrizDestino = np.zeros((2,1),dtype=np.complex_)
    
    ## Função-Objetivo (Return) = somatorio ||(Matriz_Origem - Matriz_Destino)||^2
    
    ## Avanço temporal
    
    matrizDestino[0][0] = ondaDestino[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*tempo)
    matrizDestino[1][0] = ondaDestino[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*tempo)

    y += (np.linalg.norm(matrizOrigem-matrizDestino))**2

    avanco = rungeKutta(matrizOrigem, fatorRungeKutta, somar(hamiltoniano, controles), tempoFinal = tempoFinalAnalise, tempoInicial = tempoInicialAnalise)

    matrizOrigem = avanco

    tempoInicialAnalise += 0.01
    tempoFinalAnalise += 0.01
    
    for indice in range((iteracao+1), (horizonteAnalisavel+iteracao)):
        tempo = indice/100
        
        matrizDestino[0][0] = ondaDestino[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*tempo)
        matrizDestino[1][0] = ondaDestino[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*tempo)
        
        y += (np.linalg.norm(matrizOrigem-matrizDestino))**2
        
        controles = [[0, x[int(tempoInicialAnalise*100)]], [x[int(tempoInicialAnalise*100)], 0]]

        avanco = rungeKutta(matrizOrigem, fatorRungeKutta, somar(hamiltoniano, controles), tempoFinal = tempoFinalAnalise, tempoInicial = tempoInicialAnalise)

        matrizOrigem = avanco

        tempoInicialAnalise += 0.01
        tempoFinalAnalise += 0.01
    
    return y

Para o funcionamento do MPC, cria-se um looping que consiste em uma otimização não linear, com o apoio da biblioteca de otimização do pyhton, de forma a obter o melhor resultado para o ajuste da curva. Com esse valor em mãos, utiliza-se para os pontos da próxima iteração do método de controle além de efetivar o ajuste realizando o Runge-Kutta com o valor otimizado.

In [3]:
## Constantes

horizonte = 3
fatorRungeKutta = 0.01

## Chute inicial

x = []
x.append(random.uniform(-1, 1))
for l in range(1, horizonte):
    x.append(x[0])

# Função de origem e destino

ondaOrigem = [[complex(0.80,0.0)],[complex(0.60,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

# Análise das respostas

minimos3 = np.zeros((1,2000), dtype=np.complex_) # Verificar se o mínimo será zero
constantes3 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1
valoresOnda3 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar as componentes de cada onda
valoresFuncaoObjetivo3 = np.zeros((1,2000), dtype=np.complex_) # Conferir se o mínimo está realmente ajustado

# Hamiltoniano

hamiltoniano = np.zeros((2,2), dtype=np.complex_) 
hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]

iteracao = 0
tempoAnalise = 2000

while iteracao < tempoAnalise:
    
    print(f"Iteração: {iteracao+1}")
    
    # Variação do Psi
    valoresOnda3[0][iteracao] = ondaOrigem[0][0]
    valoresOnda3[1][iteracao] = ondaOrigem[1][0]
    
    # Obtenção das constantes que multiplicam Psi
    constantes3[0][iteracao] = ondaOrigem[0][0] / np.exp(-1*complex(0,1)*(1/2*np.pi)*iteracao/100)
    constantes3[1][iteracao] = ondaOrigem[1][0] / np.exp(-3*complex(0,1)*(1/2*np.pi)*iteracao/100)
        
    ## Minimização
    
    resultado = optimize.minimize(funcaoObjetivo, x, args=(iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise), method='CG')

    print(f"Mínimo: {resultado.x}")
    
    # Função objetivo
    
    minimos3[0][iteracao] = resultado.x[0]
    
    valoresFuncaoObjetivo3[0][iteracao] = funcaoObjetivo(resultado.x, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    
    print(f" Conferindo: {valoresFuncaoObjetivo3[0][iteracao]}")
    
    ## Hamiltoniano corntrolado
    
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, resultado.x[0]],[resultado.x[0], 0]]
    hamiltoniano = somar(hamiltoniano, controle)
    
    ## Adaptação da onda ao controle
    
    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)
    
    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]
    
    # Reset
    hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]
    
    x = []
    for l in range(0, horizonte):
        x.append(resultado.x[l])
    
    iteracao += 1

Iteração: 1
Mínimo: [-0.18354231  0.80675121 -0.05984689]
 Conferindo: (0.06025365628174177+0j)
Iteração: 2
Mínimo: [ 0.41010616  0.89511899 -0.05984689]
 Conferindo: (0.06015259431871552+0j)
Iteração: 3
Mínimo: [ 0.42992122  0.90484663 -0.05984689]
 Conferindo: (0.0599810205172638+0j)
Iteração: 4
Mínimo: [ 0.44569114  0.91104782 -0.05984689]
 Conferindo: (0.05980157140620837+0j)
Iteração: 5
Mínimo: [ 0.44569114  0.91104782 -0.05984689]
 Conferindo: (0.05961918167984266+0j)
Iteração: 6
Mínimo: [ 0.44569114  0.91104782 -0.05984689]
 Conferindo: (0.05943685483903449+0j)
Iteração: 7
Mínimo: [ 0.44569114  0.91104782 -0.05984689]
 Conferindo: (0.059254784585833414+0j)
Iteração: 8
Mínimo: [ 0.44569114  0.91104782 -0.05984689]
 Conferindo: (0.05907316670275565+0j)
Iteração: 9
Mínimo: [ 0.44569114  0.91104782 -0.05984689]
 Conferindo: (0.05889219885186477+0j)
Iteração: 10
Mínimo: [ 0.44569114  0.91104782 -0.05984689]
 Conferindo: (0.05871208037192352+0j)
Iteração: 11
Mínimo: [ 0.44569114  0.91

 Conferindo: (0.04635852615052635+0j)
Iteração: 92
Mínimo: [ 0.36747134  0.77336985 -0.05984689]
 Conferindo: (0.046237980121131304+0j)
Iteração: 93
Mínimo: [ 0.36747134  0.77336985 -0.05984689]
 Conferindo: (0.04611883715382932+0j)
Iteração: 94
Mínimo: [ 0.35097426  0.75774104 -0.05984689]
 Conferindo: (0.0460010903977643+0j)
Iteração: 95
Mínimo: [ 0.35097426  0.75774104 -0.05984689]
 Conferindo: (0.045884195861390475+0j)
Iteração: 96
Mínimo: [ 0.35097426  0.75774104 -0.05984689]
 Conferindo: (0.045765888147088776+0j)
Iteração: 97
Mínimo: [ 0.35097426  0.75774104 -0.05984689]
 Conferindo: (0.04564640026835334+0j)
Iteração: 98
Mínimo: [ 0.35097426  0.75774104 -0.05984689]
 Conferindo: (0.04552596733580264+0j)
Iteração: 99
Mínimo: [ 0.35097426  0.75774104 -0.05984689]
 Conferindo: (0.045404826319379886+0j)
Iteração: 100
Mínimo: [ 0.37751498  0.7580798  -0.05984689]
 Conferindo: (0.045283074677175815+0j)
Iteração: 101
Mínimo: [ 0.36400725  0.74387797 -0.05984689]
 Conferindo: (0.04516280

Mínimo: [ 0.26641616  0.58434757 -0.05984689]
 Conferindo: (0.03833646720487009+0j)
Iteração: 177
Mínimo: [ 0.26641616  0.58434757 -0.05984689]
 Conferindo: (0.038265591070965686+0j)
Iteração: 178
Mínimo: [ 0.26641616  0.58434757 -0.05984689]
 Conferindo: (0.03819470224514404+0j)
Iteração: 179
Mínimo: [ 0.26641616  0.58434757 -0.05984689]
 Conferindo: (0.038123971306936746+0j)
Iteração: 180
Mínimo: [ 0.26641616  0.58434757 -0.05984689]
 Conferindo: (0.03805356897816063+0j)
Iteração: 181
Mínimo: [ 0.26641616  0.58434757 -0.05984689]
 Conferindo: (0.037983665952017596+0j)
Iteração: 182
Mínimo: [ 0.26641616  0.58434757 -0.05984689]
 Conferindo: (0.03791443272222173+0j)
Iteração: 183
Mínimo: [ 0.26641616  0.58434757 -0.05984689]
 Conferindo: (0.03784603941232739+0j)
Iteração: 184
Mínimo: [ 0.26641616  0.58434757 -0.05984689]
 Conferindo: (0.037778655605426925+0j)
Iteração: 185
Mínimo: [ 0.26641616  0.58434757 -0.05984689]
 Conferindo: (0.03771245017438986+0j)
Iteração: 186
Mínimo: [ 0.2664

Mínimo: [ 0.16868257  0.38863747 -0.05984689]
 Conferindo: (0.03371395255993334+0j)
Iteração: 274
Mínimo: [ 0.16868257  0.38863747 -0.05984689]
 Conferindo: (0.03368515954753073+0j)
Iteração: 275
Mínimo: [ 0.16868257  0.38863747 -0.05984689]
 Conferindo: (0.03365557292624757+0j)
Iteração: 276
Mínimo: [ 0.16868257  0.38863747 -0.05984689]
 Conferindo: (0.03362527015761693+0j)
Iteração: 277
Mínimo: [ 0.18277062  0.3903899  -0.05984689]
 Conferindo: (0.03359422655047704+0j)
Iteração: 278
Mínimo: [ 0.18277062  0.3903899  -0.05984689]
 Conferindo: (0.03356302041055401+0j)
Iteração: 279
Mínimo: [ 0.18277062  0.3903899  -0.05984689]
 Conferindo: (0.033532831452303596+0j)
Iteração: 280
Mínimo: [ 0.18277062  0.3903899  -0.05984689]
 Conferindo: (0.03350374637763471+0j)
Iteração: 281
Mínimo: [ 0.1682567   0.37673725 -0.05984689]
 Conferindo: (0.03347565995898538+0j)
Iteração: 282
Mínimo: [ 0.1682567   0.37673725 -0.05984689]
 Conferindo: (0.03344840025998138+0j)
Iteração: 283
Mínimo: [ 0.1682567

Mínimo: [ 0.1347794   0.27804994 -0.05984689]
 Conferindo: (0.031696823933232796+0j)
Iteração: 371
Mínimo: [ 0.1347794   0.27804994 -0.05984689]
 Conferindo: (0.031682536638772726+0j)
Iteração: 372
Mínimo: [ 0.11136415  0.25915908 -0.05984689]
 Conferindo: (0.03166959173352339+0j)
Iteração: 373
Mínimo: [ 0.11136415  0.25915908 -0.05984689]
 Conferindo: (0.03165785653569799+0j)
Iteração: 374
Mínimo: [ 0.11136415  0.25915908 -0.05984689]
 Conferindo: (0.03164610003404944+0j)
Iteração: 375
Mínimo: [ 0.11136415  0.25915908 -0.05984689]
 Conferindo: (0.03163435635489267+0j)
Iteração: 376
Mínimo: [ 0.11136415  0.25915908 -0.05984689]
 Conferindo: (0.03162265962104417+0j)
Iteração: 377
Mínimo: [ 0.11136415  0.25915908 -0.05984689]
 Conferindo: (0.031611043918063284+0j)
Iteração: 378
Mínimo: [ 0.11136415  0.25915908 -0.05984689]
 Conferindo: (0.031599543260529425+0j)
Iteração: 379
Mínimo: [ 0.11136415  0.25915908 -0.05984689]
 Conferindo: (0.03158819155838973+0j)
Iteração: 380
Mínimo: [ 0.1113

...

Mínimo: [ 0.01029935  0.00613065 -0.05984689]
 Conferindo: (0.030303833332770664+0j)
Iteração: 1613
Mínimo: [ 0.01029935  0.00613065 -0.05984689]
 Conferindo: (0.0303039054558377+0j)
Iteração: 1614
Mínimo: [-0.00680539 -0.00319401 -0.05984689]
 Conferindo: (0.030303860243853616+0j)
Iteração: 1615
Mínimo: [-0.00680539 -0.00319401 -0.05984689]
 Conferindo: (0.030303823576963096+0j)
Iteração: 1616
Mínimo: [-0.00680539 -0.00319401 -0.05984689]
 Conferindo: (0.030303815683723964+0j)
Iteração: 1617
Mínimo: [-0.00680539 -0.00319401 -0.05984689]
 Conferindo: (0.0303038366589519+0j)
Iteração: 1618
Mínimo: [-0.00680539 -0.00319401 -0.05984689]
 Conferindo: (0.03030388656897534+0j)
Iteração: 1619
Mínimo: [ 0.00722546  0.00434263 -0.05984689]
 Conferindo: (0.03030384557705581+0j)
Iteração: 1620
Mínimo: [ 0.00722546  0.00434263 -0.05984689]
 Conferindo: (0.03030381142494036+0j)
Iteração: 1621
Mínimo: [ 0.00722546  0.00434263 -0.05984689]
 Conferindo: (0.030303807278956353+0j)
Iteração: 1622
Mínimo:

 Conferindo: (0.030303743516852572+0j)
Iteração: 1697
Mínimo: [-0.00534376 -0.00306243 -0.05984689]
 Conferindo: (0.030303770305707144+0j)
Iteração: 1698
Mínimo: [-0.00534376 -0.00306243 -0.05984689]
 Conferindo: (0.030303814648833034+0j)
Iteração: 1699
Mínimo: [-0.00534376 -0.00306243 -0.05984689]
 Conferindo: (0.030303876555915994+0j)
Iteração: 1700
Mínimo: [ 0.0110014   0.00556558 -0.05984689]
 Conferindo: (0.03030379453228777+0j)
Iteração: 1701
Mínimo: [ 0.0110014   0.00556558 -0.05984689]
 Conferindo: (0.030303737524654036+0j)
Iteração: 1702
Mínimo: [ 0.0110014   0.00556558 -0.05984689]
 Conferindo: (0.030303752267239976+0j)
Iteração: 1703
Mínimo: [ 0.0110014   0.00556558 -0.05984689]
 Conferindo: (0.030303838973897483+0j)
Iteração: 1704
Mínimo: [-0.00767207 -0.00452152 -0.05984689]
 Conferindo: (0.030303784959995087+0j)
Iteração: 1705
Mínimo: [-0.00767207 -0.00452152 -0.05984689]
 Conferindo: (0.030303738328314865+0j)
Iteração: 1706
Mínimo: [-0.00767207 -0.00452152 -0.05984689]
 

Mínimo: [ 0.01003016  0.00592598 -0.05984689]
 Conferindo: (0.030303731591946118+0j)
Iteração: 1790
Mínimo: [ 0.01003016  0.00592598 -0.05984689]
 Conferindo: (0.03030367217023098+0j)
Iteração: 1791
Mínimo: [ 0.01003016  0.00592598 -0.05984689]
 Conferindo: (0.030303672137751633+0j)
Iteração: 1792
Mínimo: [ 0.01003016  0.00592598 -0.05984689]
 Conferindo: (0.030303731684286656+0j)
Iteração: 1793
Mínimo: [-0.00592218 -0.00283996 -0.05984689]
 Conferindo: (0.030303694466059745+0j)
Iteração: 1794
Mínimo: [-0.00592218 -0.00283996 -0.05984689]
 Conferindo: (0.030303664114048275+0j)
Iteração: 1795
Mínimo: [-0.00592218 -0.00283996 -0.05984689]
 Conferindo: (0.030303655068807476+0j)
Iteração: 1796
Mínimo: [-0.00592218 -0.00283996 -0.05984689]
 Conferindo: (0.030303667405049188+0j)
Iteração: 1797
Mínimo: [-0.00592218 -0.00283996 -0.05984689]
 Conferindo: (0.030303701176379275+0j)
Iteração: 1798
Mínimo: [-0.00592218 -0.00283996 -0.05984689]
 Conferindo: (0.030303756415256483+0j)
Iteração: 1799
M

Mínimo: [ 0.00505542  0.00245037 -0.05984689]
 Conferindo: (0.030303621259585127+0j)
Iteração: 1873
Mínimo: [ 0.00505542  0.00245037 -0.05984689]
 Conferindo: (0.030303659906990514+0j)
Iteração: 1874
Mínimo: [ 0.00505542  0.00245037 -0.05984689]
 Conferindo: (0.03030371394222891+0j)
Iteração: 1875
Mínimo: [-0.01006234 -0.00544794 -0.05984689]
 Conferindo: (0.030303645770889183+0j)
Iteração: 1876
Mínimo: [-0.01006234 -0.00544794 -0.05984689]
 Conferindo: (0.03030359731995433+0j)
Iteração: 1877
Mínimo: [-0.01006234 -0.00544794 -0.05984689]
 Conferindo: (0.030303609526661772+0j)
Iteração: 1878
Mínimo: [-0.01006234 -0.00544794 -0.05984689]
 Conferindo: (0.03030368256991316+0j)
Iteração: 1879
Mínimo: [ 0.00716952  0.0039317  -0.05984689]
 Conferindo: (0.030303634722180773+0j)
Iteração: 1880
Mínimo: [ 0.00716952  0.0039317  -0.05984689]
 Conferindo: (0.03030359442214241+0j)
Iteração: 1881
Mínimo: [ 0.00716952  0.0039317  -0.05984689]
 Conferindo: (0.03030358468575718+0j)
Iteração: 1882
Mínim

Mínimo: [-0.00923952 -0.00450054 -0.05984689]
 Conferindo: (0.030303537003746495+0j)
Iteração: 1956
Mínimo: [-0.00923952 -0.00450054 -0.05984689]
 Conferindo: (0.03030353727604572+0j)
Iteração: 1957
Mínimo: [-0.00923952 -0.00450054 -0.05984689]
 Conferindo: (0.030303589162346222+0j)
Iteração: 1958
Mínimo: [ 0.00563162  0.00358401 -0.05984689]
 Conferindo: (0.03030355743050471+0j)
Iteração: 1959
Mínimo: [ 0.00563162  0.00358401 -0.05984689]
 Conferindo: (0.030303529299947896+0j)
Iteração: 1960
Mínimo: [ 0.00563162  0.00358401 -0.05984689]
 Conferindo: (0.030303519792735996+0j)
Iteração: 1961
Mínimo: [ 0.00563162  0.00358401 -0.05984689]
 Conferindo: (0.030303528977649763+0j)
Iteração: 1962
Mínimo: [ 0.00563162  0.00358401 -0.05984689]
 Conferindo: (0.030303556905025425+0j)
Iteração: 1963
Mínimo: [ 0.00563162  0.00358401 -0.05984689]
 Conferindo: (0.030303603606698988+0j)
Iteração: 1964
Mínimo: [ 0.00563162  0.00358401 -0.05984689]
 Conferindo: (0.03030366909598688+0j)
Iteração: 1965
Mín

In [4]:
## Constantes

horizonte = 4
fatorRungeKutta = 0.01

## Chute inicial

x = []
x.append(random.uniform(-1, 1))
for l in range(1, horizonte):
    x.append(x[0])

# Função de origem e destino

ondaOrigem = [[complex(0.80,0.0)],[complex(0.60,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

# Análise das respostas

minimos4 = np.zeros((1,2000), dtype=np.complex_) # Verificar se o mínimo será zero
constantes4 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1
valoresOnda4 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar as componentes de cada onda
valoresFuncaoObjetivo4 = np.zeros((1,2000), dtype=np.complex_) # Conferir se o mínimo está realmente ajustado

# Hamiltoniano

hamiltoniano = np.zeros((2,2), dtype=np.complex_) 
hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]

iteracao = 0
tempoAnalise = 2000

while iteracao < tempoAnalise:
    
    print(f"Iteração: {iteracao+1}")
    
    # Variação do Psi
    valoresOnda4[0][iteracao] = ondaOrigem[0][0]
    valoresOnda4[1][iteracao] = ondaOrigem[1][0]
    
    # Obtenção das constantes que multiplicam Psi
    constantes4[0][iteracao] = ondaOrigem[0][0] / np.exp(-1*complex(0,1)*(1/2*np.pi)*iteracao/100)
    constantes4[1][iteracao] = ondaOrigem[1][0] / np.exp(-3*complex(0,1)*(1/2*np.pi)*iteracao/100)
        
    ## Minimização
    
    resultado = optimize.minimize(funcaoObjetivo, x, args=(iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise), method='CG')

    print(f"Mínimo: {resultado.x}")
    
    # Função objetivo
    
    minimos4[0][iteracao] = resultado.x[0]
    
    valoresFuncaoObjetivo4[0][iteracao] = funcaoObjetivo(resultado.x, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    
    print(f" Conferindo: {valoresFuncaoObjetivo4[0][iteracao]}")
    
    ## Hamiltoniano corntrolado
    
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, resultado.x[0]],[resultado.x[0], 0]]
    hamiltoniano = somar(hamiltoniano, controle)
    
    ## Adaptação da onda ao controle
    
    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)
    
    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]
    
    # Reset
    hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]
    
    x = []
    for l in range(0, horizonte):
        x.append(resultado.x[l])
    
    iteracao += 1

Iteração: 1
Mínimo: [-0.61160351  0.81576238  0.9228461  -0.36401872]
 Conferindo: (0.08023054058234244+0j)
Iteração: 2
Mínimo: [ 0.34068286  0.97886312  0.86394345 -0.36401872]
 Conferindo: (0.08001324063794982+0j)
Iteração: 3
Mínimo: [ 0.46051531  0.95971323  0.86040983 -0.36401872]
 Conferindo: (0.07962883314688168+0j)
Iteração: 4
Mínimo: [ 0.45340258  0.94604585  0.85671121 -0.36401872]
 Conferindo: (0.07921832346859281+0j)
Iteração: 5
Mínimo: [ 0.45340258  0.94604585  0.85671121 -0.36401872]
 Conferindo: (0.07881339757307729+0j)
Iteração: 6
Mínimo: [ 0.44590806  0.93378381  0.85346963 -0.36401872]
 Conferindo: (0.07841239940622025+0j)
Iteração: 7
Mínimo: [ 0.44590806  0.93378381  0.85346963 -0.36401872]
 Conferindo: (0.0780152009022056+0j)
Iteração: 8
Mínimo: [ 0.44590806  0.93378381  0.85346963 -0.36401872]
 Conferindo: (0.07762015124453571+0j)
Iteração: 9
Mínimo: [ 0.44590806  0.93378381  0.85346963 -0.36401872]
 Conferindo: (0.07722766752748587+0j)
Iteração: 10
Mínimo: [ 0.4361

Mínimo: [ 0.34686261  0.72112255  0.71552134 -0.36401872]
 Conferindo: (0.05319667983330847+0j)
Iteração: 86
Mínimo: [ 0.33810373  0.70523019  0.702653   -0.36401872]
 Conferindo: (0.05295696163106051+0j)
Iteração: 87
Mínimo: [ 0.33810373  0.70523019  0.702653   -0.36401872]
 Conferindo: (0.05271962991155155+0j)
Iteração: 88
Mínimo: [ 0.33810373  0.70523019  0.702653   -0.36401872]
 Conferindo: (0.05248303941229495+0j)
Iteração: 89
Mínimo: [ 0.33810373  0.70523019  0.702653   -0.36401872]
 Conferindo: (0.05224754812790136+0j)
Iteração: 90
Mínimo: [ 0.33810373  0.70523019  0.702653   -0.36401872]
 Conferindo: (0.05201351398147663+0j)
Iteração: 91
Mínimo: [ 0.33810373  0.70523019  0.702653   -0.36401872]
 Conferindo: (0.051781294463243546+0j)
Iteração: 92
Mínimo: [ 0.3324886   0.68973931  0.68887449 -0.36401872]
 Conferindo: (0.051551076703043285+0j)
Iteração: 93
Mínimo: [ 0.3324886   0.68973931  0.68887449 -0.36401872]
 Conferindo: (0.05132315111882826+0j)
Iteração: 94
Mínimo: [ 0.33248

Mínimo: [ 0.23875741  0.49682668  0.51316003 -0.36401872]
 Conferindo: (0.03889668553368024+0j)
Iteração: 167
Mínimo: [ 0.23875741  0.49682668  0.51316003 -0.36401872]
 Conferindo: (0.03877654052657306+0j)
Iteração: 168
Mínimo: [ 0.23875741  0.49682668  0.51316003 -0.36401872]
 Conferindo: (0.038656770408824+0j)
Iteração: 169
Mínimo: [ 0.23875741  0.49682668  0.51316003 -0.36401872]
 Conferindo: (0.03853758466314358+0j)
Iteração: 170
Mínimo: [ 0.23875741  0.49682668  0.51316003 -0.36401872]
 Conferindo: (0.03841919244758457+0j)
Iteração: 171
Mínimo: [ 0.23875741  0.49682668  0.51316003 -0.36401872]
 Conferindo: (0.03830180238674864+0j)
Iteração: 172
Mínimo: [ 0.23875741  0.49682668  0.51316003 -0.36401872]
 Conferindo: (0.03818562236352581+0j)
Iteração: 173
Mínimo: [ 0.23208502  0.4823869   0.49916483 -0.36401872]
 Conferindo: (0.03807068777073296+0j)
Iteração: 174
Mínimo: [ 0.23208502  0.4823869   0.49916483 -0.36401872]
 Conferindo: (0.03795721730155816+0j)
Iteração: 175
Mínimo: [ 0.

Mínimo: [ 0.1683595   0.35440277  0.37524533 -0.36401872]
 Conferindo: (0.03206169830336994+0j)
Iteração: 246
Mínimo: [ 0.16345754  0.34126753  0.36155581 -0.36401872]
 Conferindo: (0.032003686599981665+0j)
Iteração: 247
Mínimo: [ 0.16345754  0.34126753  0.36155581 -0.36401872]
 Conferindo: (0.03194661131303263+0j)
Iteração: 248
Mínimo: [ 0.16345754  0.34126753  0.36155581 -0.36401872]
 Conferindo: (0.03188993930370941+0j)
Iteração: 249
Mínimo: [ 0.16345754  0.34126753  0.36155581 -0.36401872]
 Conferindo: (0.03183377583305386+0j)
Iteração: 250
Mínimo: [ 0.16345754  0.34126753  0.36155581 -0.36401872]
 Conferindo: (0.031778225714400095+0j)
Iteração: 251
Mínimo: [ 0.16345754  0.34126753  0.36155581 -0.36401872]
 Conferindo: (0.03172339320937671+0j)
Iteração: 252
Mínimo: [ 0.16345754  0.34126753  0.36155581 -0.36401872]
 Conferindo: (0.03166938192445431+0j)
Iteração: 253
Mínimo: [ 0.15433877  0.32725911  0.34877482 -0.36401872]
 Conferindo: (0.03161610419689846+0j)
Iteração: 254
Mínimo: 

Mínimo: [ 0.11204248  0.2430884   0.26604846 -0.36401872]
 Conferindo: (0.028962935755501656+0j)
Iteração: 321
Mínimo: [ 0.11204248  0.2430884   0.26604846 -0.36401872]
 Conferindo: (0.02893450731956979+0j)
Iteração: 322
Mínimo: [ 0.11204248  0.2430884   0.26604846 -0.36401872]
 Conferindo: (0.028906324038022888+0j)
Iteração: 323
Mínimo: [ 0.11204248  0.2430884   0.26604846 -0.36401872]
 Conferindo: (0.02887843821090258+0j)
Iteração: 324
Mínimo: [ 0.11204248  0.2430884   0.26604846 -0.36401872]
 Conferindo: (0.028850901856499696+0j)
Iteração: 325
Mínimo: [ 0.11204248  0.2430884   0.26604846 -0.36401872]
 Conferindo: (0.028823766659888794+0j)
Iteração: 326
Mínimo: [ 0.11204248  0.2430884   0.26604846 -0.36401872]
 Conferindo: (0.028797083921791557+0j)
Iteração: 327
Mínimo: [ 0.10710302  0.22972274  0.25203287 -0.36401872]
 Conferindo: (0.028770751958858944+0j)
Iteração: 328
Mínimo: [ 0.10710302  0.22972274  0.25203287 -0.36401872]
 Conferindo: (0.028745016484570517+0j)
Iteração: 329
Mín

...

 Conferindo: (0.02641009972653749+0j)
Iteração: 1619
Mínimo: [-0.00450423 -0.00351059 -0.00183426 -0.36401872]
 Conferindo: (0.026410131853539916+0j)
Iteração: 1620
Mínimo: [ 0.00615128  0.00423365  0.00224662 -0.36401872]
 Conferindo: (0.026410085663993662+0j)
Iteração: 1621
Mínimo: [ 0.00615128  0.00423365  0.00224662 -0.36401872]
 Conferindo: (0.02641008428658064+0j)
Iteração: 1622
Mínimo: [ 0.00615128  0.00423365  0.00224662 -0.36401872]
 Conferindo: (0.026410112574740116+0j)
Iteração: 1623
Mínimo: [-0.00424847 -0.00336969 -0.00175917 -0.36401872]
 Conferindo: (0.026410079066980445+0j)
Iteração: 1624
Mínimo: [-0.00424847 -0.00336969 -0.00175917 -0.36401872]
 Conferindo: (0.026410077857669287+0j)
Iteração: 1625
Mínimo: [-0.00424847 -0.00336969 -0.00175917 -0.36401872]
 Conferindo: (0.02641009144697449+0j)
Iteração: 1626
Mínimo: [-0.00424847 -0.00336969 -0.00175917 -0.36401872]
 Conferindo: (0.026410119857667703+0j)
Iteração: 1627
Mínimo: [ 0.00578801  0.00393554  0.00209246 -0.36401

Mínimo: [-0.00488986 -0.00379328 -0.00202063 -0.36401872]
 Conferindo: (0.02641008403732835+0j)
Iteração: 1699
Mínimo: [ 0.00654085  0.00450045  0.0023448  -0.36401872]
 Conferindo: (0.026410031137369363+0j)
Iteração: 1700
Mínimo: [ 0.00654085  0.00450045  0.0023448  -0.36401872]
 Conferindo: (0.02641002957664312+0j)
Iteração: 1701
Mínimo: [ 0.00654085  0.00450045  0.0023448  -0.36401872]
 Conferindo: (0.02641006180846873+0j)
Iteração: 1702
Mínimo: [-0.00456564 -0.00363365 -0.00194439 -0.36401872]
 Conferindo: (0.026410023530836504+0j)
Iteração: 1703
Mínimo: [-0.00456564 -0.00363365 -0.00194439 -0.36401872]
 Conferindo: (0.02641002210303288+0j)
Iteração: 1704
Mínimo: [-0.00456564 -0.00363365 -0.00194439 -0.36401872]
 Conferindo: (0.026410037587815495+0j)
Iteração: 1705
Mínimo: [-0.00456564 -0.00363365 -0.00194439 -0.36401872]
 Conferindo: (0.0264100700118188+0j)
Iteração: 1706
Mínimo: [ 0.00609239  0.00411448  0.0021391  -0.36401872]
 Conferindo: (0.026410023758597204+0j)
Iteração: 170

Iteração: 1773
Mínimo: [ 0.00765171  0.00595121  0.00324438 -0.36401872]
 Conferindo: (0.026410030258669916+0j)
Iteração: 1774
Mínimo: [-0.00532721 -0.0036801  -0.00188187 -0.36401872]
 Conferindo: (0.02640997540218128+0j)
Iteração: 1775
Mínimo: [-0.00532721 -0.0036801  -0.00188187 -0.36401872]
 Conferindo: (0.026409974556259923+0j)
Iteração: 1776
Mínimo: [-0.00532721 -0.0036801  -0.00188187 -0.36401872]
 Conferindo: (0.02640999670704139+0j)
Iteração: 1777
Mínimo: [-0.00532721 -0.0036801  -0.00188187 -0.36401872]
 Conferindo: (0.026410041889977268+0j)
Iteração: 1778
Mínimo: [ 0.00719711  0.00531963  0.00282277 -0.36401872]
 Conferindo: (0.02640998040411566+0j)
Iteração: 1779
Mínimo: [ 0.00719711  0.00531963  0.00282277 -0.36401872]
 Conferindo: (0.026409978270914555+0j)
Iteração: 1780
Mínimo: [ 0.00719711  0.00531963  0.00282277 -0.36401872]
 Conferindo: (0.02641001713386952+0j)
Iteração: 1781
Mínimo: [-0.0050451  -0.00371549 -0.0019662  -0.36401872]
 Conferindo: (0.026409969307351424+

Mínimo: [-0.00468552 -0.00365513 -0.00198071 -0.36401872]
 Conferindo: (0.026409917778513967+0j)
Iteração: 1853
Mínimo: [-0.00468552 -0.00365513 -0.00198071 -0.36401872]
 Conferindo: (0.026409916376515435+0j)
Iteração: 1854
Mínimo: [-0.00468552 -0.00365513 -0.00198071 -0.36401872]
 Conferindo: (0.026409932640735663+0j)
Iteração: 1855
Mínimo: [-0.00468552 -0.00365513 -0.00198071 -0.36401872]
 Conferindo: (0.02640996659926418+0j)
Iteração: 1856
Mínimo: [ 0.00620097  0.00423439  0.00216979 -0.36401872]
 Conferindo: (0.026409918909319163+0j)
Iteração: 1857
Mínimo: [ 0.00620097  0.00423439  0.00216979 -0.36401872]
 Conferindo: (0.0264099177630592+0j)
Iteração: 1858
Mínimo: [ 0.00620097  0.00423439  0.00216979 -0.36401872]
 Conferindo: (0.026409947197662327+0j)
Iteração: 1859
Mínimo: [-0.00440113 -0.00353403 -0.00192832 -0.36401872]
 Conferindo: (0.0264099120265677+0j)
Iteração: 1860
Mínimo: [-0.00440113 -0.00353403 -0.00192832 -0.36401872]
 Conferindo: (0.026409910434412202+0j)
Iteração: 18

Mínimo: [-0.00523027 -0.00454416 -0.00247429 -0.36401872]
 Conferindo: (0.02640986313726191+0j)
Iteração: 1933
Mínimo: [-0.00523027 -0.00454416 -0.00247429 -0.36401872]
 Conferindo: (0.02640988196097908+0j)
Iteração: 1934
Mínimo: [-0.00523027 -0.00454416 -0.00247429 -0.36401872]
 Conferindo: (0.026409922472667224+0j)
Iteração: 1935
Mínimo: [ 0.0066659   0.00413963  0.00210612 -0.36401872]
 Conferindo: (0.02640986540562996+0j)
Iteração: 1936
Mínimo: [ 0.0066659   0.00413963  0.00210612 -0.36401872]
 Conferindo: (0.026409865401983968+0j)
Iteração: 1937
Mínimo: [ 0.0066659   0.00413963  0.00210612 -0.36401872]
 Conferindo: (0.02640990117413732+0j)
Iteração: 1938
Mínimo: [-0.00490337 -0.00428836 -0.00233039 -0.36401872]
 Conferindo: (0.026409859844216135+0j)
Iteração: 1939
Mínimo: [-0.00490337 -0.00428836 -0.00233039 -0.36401872]
 Conferindo: (0.026409857180518113+0j)
Iteração: 1940
Mínimo: [-0.00490337 -0.00428836 -0.00233039 -0.36401872]
 Conferindo: (0.026409873520511933+0j)
Iteração: 1

In [5]:
## Constantes

horizonte = 5
fatorRungeKutta = 0.01

## Chute inicial

x = []
x.append(random.uniform(-1, 1))
for l in range(1, horizonte):
    x.append(x[0])

# Função de origem e destino

ondaOrigem = [[complex(0.80,0.0)],[complex(0.60,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

# Análise das respostas

minimos5 = np.zeros((1,2000), dtype=np.complex_) # Verificar se o mínimo será zero
constantes5 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1
valoresOnda5 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar as componentes de cada onda
valoresFuncaoObjetivo5 = np.zeros((1,2000), dtype=np.complex_) # Conferir se o mínimo está realmente ajustado

# Hamiltoniano

hamiltoniano = np.zeros((2,2), dtype=np.complex_) 
hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]

iteracao = 0
tempoAnalise = 2000

while iteracao < tempoAnalise:
    
    print(f"Iteração: {iteracao+1}")
    
    # Variação do Psi
    valoresOnda5[0][iteracao] = ondaOrigem[0][0]
    valoresOnda5[1][iteracao] = ondaOrigem[1][0]
    
    # Obtenção das constantes que multiplicam Psi
    constantes5[0][iteracao] = ondaOrigem[0][0] / np.exp(-1*complex(0,1)*(1/2*np.pi)*iteracao/100)
    constantes5[1][iteracao] = ondaOrigem[1][0] / np.exp(-3*complex(0,1)*(1/2*np.pi)*iteracao/100)
        
    ## Minimização
    
    resultado = optimize.minimize(funcaoObjetivo, x, args=(iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise), method='CG')

    print(f"Mínimo: {resultado.x}")
    
    # Função objetivo
    
    minimos5[0][iteracao] = resultado.x[0]
    
    valoresFuncaoObjetivo5[0][iteracao] = funcaoObjetivo(resultado.x, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    
    print(f" Conferindo: {valoresFuncaoObjetivo5[0][iteracao]}")
    
    ## Hamiltoniano corntrolado
    
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, resultado.x[0]],[resultado.x[0], 0]]
    hamiltoniano = somar(hamiltoniano, controle)
    
    ## Adaptação da onda ao controle
    
    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)
    
    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]
    
    # Reset
    hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]
    
    x = []
    for l in range(0, horizonte):
        x.append(resultado.x[l])
    
    iteracao += 1

Iteração: 1
Mínimo: [-1.10573206  0.85789487  0.96274582  0.82872588  0.68685672]
 Conferindo: (0.10008919075770617+0j)
Iteração: 2
Mínimo: [0.41563221 0.92184699 0.88319342 0.94324487 0.68685672]
 Conferindo: (0.09971075644405088+0j)
Iteração: 3
Mínimo: [0.43496005 0.92933795 0.88496523 0.93879826 0.68685672]
 Conferindo: (0.09902383796319555+0j)
Iteração: 4
Mínimo: [0.43496005 0.92933795 0.88496523 0.93879826 0.68685672]
 Conferindo: (0.09832737448618661+0j)
Iteração: 5
Mínimo: [0.50168677 0.91468598 0.84883811 0.88544298 0.68685672]
 Conferindo: (0.09762824860155303+0j)
Iteração: 6
Mínimo: [0.46973808 0.88881461 0.83347115 0.87713908 0.68685672]
 Conferindo: (0.09693368359114615+0j)
Iteração: 7
Mínimo: [0.45240193 0.87699126 0.82884871 0.87450151 0.68685672]
 Conferindo: (0.0962650748272873+0j)
Iteração: 8
Mínimo: [0.44081132 0.87089982 0.82891062 0.874089   0.68685672]
 Conferindo: (0.09560990999464934+0j)
Iteração: 9
Mínimo: [0.44081132 0.87089982 0.82891062 0.874089   0.68685672]

Mínimo: [0.33560491 0.68487105 0.65183937 0.70576654 0.68685672]
 Conferindo: (0.06095876049248667+0j)
Iteração: 75
Mínimo: [0.33560491 0.68487105 0.65183937 0.70576654 0.68685672]
 Conferindo: (0.06056721298977876+0j)
Iteração: 76
Mínimo: [0.33560491 0.68487105 0.65183937 0.70576654 0.68685672]
 Conferindo: (0.06017888011062912+0j)
Iteração: 77
Mínimo: [0.33093302 0.67550379 0.64242803 0.69650298 0.68685672]
 Conferindo: (0.05979409161301183+0j)
Iteração: 78
Mínimo: [0.33093302 0.67550379 0.64242803 0.69650298 0.68685672]
 Conferindo: (0.0594133015499322+0j)
Iteração: 79
Mínimo: [0.33093302 0.67550379 0.64242803 0.69650298 0.68685672]
 Conferindo: (0.05903566060656029+0j)
Iteração: 80
Mínimo: [0.32628404 0.66618809 0.63307542 0.68726653 0.68685672]
 Conferindo: (0.058661490022466484+0j)
Iteração: 81
Mínimo: [0.32628404 0.66618809 0.63307542 0.68726653 0.68685672]
 Conferindo: (0.05829123570794641+0j)
Iteração: 82
Mínimo: [0.32628404 0.66618809 0.63307542 0.68726653 0.68685672]
 Confer

Mínimo: [0.23103069 0.47810966 0.44649525 0.498309   0.68685672]
 Conferindo: (0.0401739873653216+0j)
Iteração: 149
Mínimo: [0.23103069 0.47810966 0.44649525 0.498309   0.68685672]
 Conferindo: (0.03998543647784903+0j)
Iteração: 150
Mínimo: [0.23103069 0.47810966 0.44649525 0.498309   0.68685672]
 Conferindo: (0.03979845068618826+0j)
Iteração: 151
Mínimo: [0.23103069 0.47810966 0.44649525 0.498309   0.68685672]
 Conferindo: (0.03961330484616373+0j)
Iteração: 152
Mínimo: [0.2262721  0.46904805 0.43749853 0.48952676 0.68685672]
 Conferindo: (0.03943009982397323+0j)
Iteração: 153
Mínimo: [0.2262721  0.46904805 0.43749853 0.48952676 0.68685672]
 Conferindo: (0.03924915645866507+0j)
Iteração: 154
Mínimo: [0.2262721  0.46904805 0.43749853 0.48952676 0.68685672]
 Conferindo: (0.03906974336649797+0j)
Iteração: 155
Mínimo: [0.2262721  0.46904805 0.43749853 0.48952676 0.68685672]
 Conferindo: (0.03889212522678582+0j)
Iteração: 156
Mínimo: [0.22116979 0.45971401 0.42829224 0.48066012 0.68685672]


Mínimo: [0.15559166 0.33077516 0.30028643 0.35267274 0.68685672]
 Conferindo: (0.030531375727050506+0j)
Iteração: 222
Mínimo: [0.15559166 0.33077516 0.30028643 0.35267274 0.68685672]
 Conferindo: (0.030444583721371158+0j)
Iteração: 223
Mínimo: [0.15559166 0.33077516 0.30028643 0.35267274 0.68685672]
 Conferindo: (0.030358660014529387+0j)
Iteração: 224
Mínimo: [0.15302775 0.32256305 0.29143855 0.34327992 0.68685672]
 Conferindo: (0.030273602223547955+0j)
Iteração: 225
Mínimo: [0.15302775 0.32256305 0.29143855 0.34327992 0.68685672]
 Conferindo: (0.030189684755688874+0j)
Iteração: 226
Mínimo: [0.15302775 0.32256305 0.29143855 0.34327992 0.68685672]
 Conferindo: (0.030106613034327635+0j)
Iteração: 227
Mínimo: [0.15302775 0.32256305 0.29143855 0.34327992 0.68685672]
 Conferindo: (0.030024514573178885+0j)
Iteração: 228
Mínimo: [0.15302775 0.32256305 0.29143855 0.34327992 0.68685672]
 Conferindo: (0.029943515975594224+0j)
Iteração: 229
Mínimo: [0.14710901 0.31317596 0.2825974  0.33499199 0.6

Mínimo: [0.10298109 0.22410795 0.19434154 0.24554269 0.68685672]
 Conferindo: (0.026108166114461477+0j)
Iteração: 297
Mínimo: [0.10298109 0.22410795 0.19434154 0.24554269 0.68685672]
 Conferindo: (0.026070871788739847+0j)
Iteração: 298
Mínimo: [0.09791322 0.21569206 0.18637862 0.23782917 0.68685672]
 Conferindo: (0.02603400405802564+0j)
Iteração: 299
Mínimo: [0.09791322 0.21569206 0.18637862 0.23782917 0.68685672]
 Conferindo: (0.025997800100382083+0j)
Iteração: 300
Mínimo: [0.09791322 0.21569206 0.18637862 0.23782917 0.68685672]
 Conferindo: (0.025961762716623784+0j)
Iteração: 301
Mínimo: [0.09791322 0.21569206 0.18637862 0.23782917 0.68685672]
 Conferindo: (0.025925947118893126+0j)
Iteração: 302
Mínimo: [0.09791322 0.21569206 0.18637862 0.23782917 0.68685672]
 Conferindo: (0.025890408309178603+0j)
Iteração: 303
Mínimo: [0.09791322 0.21569206 0.18637862 0.23782917 0.68685672]
 Conferindo: (0.02585520102492876+0j)
Iteração: 304
Mínimo: [0.09791322 0.21569206 0.18637862 0.23782917 0.686

...

Mínimo: [-0.00198952 -0.00184143 -0.00134484 -0.00072976  0.68685672]
 Conferindo: (0.022859643975697702+0j)
Iteração: 1705
Mínimo: [-0.00198952 -0.00184143 -0.00134484 -0.00072976  0.68685672]
 Conferindo: (0.02285964340657233+0j)
Iteração: 1706
Mínimo: [-0.00198952 -0.00184143 -0.00134484 -0.00072976  0.68685672]
 Conferindo: (0.022859646730459288+0j)
Iteração: 1707
Mínimo: [-0.00198952 -0.00184143 -0.00134484 -0.00072976  0.68685672]
 Conferindo: (0.022859653951842022+0j)
Iteração: 1708
Mínimo: [-0.00198952 -0.00184143 -0.00134484 -0.00072976  0.68685672]
 Conferindo: (0.022859665071357237+0j)
Iteração: 1709
Mínimo: [-0.00198952 -0.00184143 -0.00134484 -0.00072976  0.68685672]
 Conferindo: (0.022859680085796037+0j)
Iteração: 1710
Mínimo: [-0.00198952 -0.00184143 -0.00134484 -0.00072976  0.68685672]
 Conferindo: (0.02285969898810318+0j)
Iteração: 1711
Mínimo: [0.00480825 0.00352022 0.00235991 0.00116053 0.68685672]
 Conferindo: (0.022859649607508012+0j)
Iteração: 1712
Mínimo: [0.0048

Mínimo: [0.00396081 0.00352892 0.00267027 0.00140479 0.68685672]
 Conferindo: (0.022859604890792737+0j)
Iteração: 1773
Mínimo: [0.00396081 0.00352892 0.00267027 0.00140479 0.68685672]
 Conferindo: (0.02285960364649646+0j)
Iteração: 1774
Mínimo: [0.00396081 0.00352892 0.00267027 0.00140479 0.68685672]
 Conferindo: (0.022859618063076105+0j)
Iteração: 1775
Mínimo: [0.00396081 0.00352892 0.00267027 0.00140479 0.68685672]
 Conferindo: (0.022859648158935906+0j)
Iteração: 1776
Mínimo: [-0.00368759 -0.00274743 -0.00179407 -0.00091386  0.68685672]
 Conferindo: (0.022859597871230465+0j)
Iteração: 1777
Mínimo: [-0.00368759 -0.00274743 -0.00179407 -0.00091386  0.68685672]
 Conferindo: (0.0228595981970203+0j)
Iteração: 1778
Mínimo: [-0.00368759 -0.00274743 -0.00179407 -0.00091386  0.68685672]
 Conferindo: (0.022859612135786772+0j)
Iteração: 1779
Mínimo: [-0.00368759 -0.00274743 -0.00179407 -0.00091386  0.68685672]
 Conferindo: (0.022859639702017168+0j)
Iteração: 1780
Mínimo: [0.00356695 0.00309172 

Mínimo: [-0.00210071 -0.00209308 -0.00158982 -0.00087646  0.68685672]
 Conferindo: (0.022859548441601818+0j)
Iteração: 1845
Mínimo: [-0.00210071 -0.00209308 -0.00158982 -0.00087646  0.68685672]
 Conferindo: (0.022859547624193496+0j)
Iteração: 1846
Mínimo: [-0.00210071 -0.00209308 -0.00158982 -0.00087646  0.68685672]
 Conferindo: (0.022859551126218382+0j)
Iteração: 1847
Mínimo: [-0.00210071 -0.00209308 -0.00158982 -0.00087646  0.68685672]
 Conferindo: (0.022859558952924817+0j)
Iteração: 1848
Mínimo: [-0.00210071 -0.00209308 -0.00158982 -0.00087646  0.68685672]
 Conferindo: (0.02285957110529477+0j)
Iteração: 1849
Mínimo: [-0.00210071 -0.00209308 -0.00158982 -0.00087646  0.68685672]
 Conferindo: (0.022859587580040697+0j)
Iteração: 1850
Mínimo: [-0.00210071 -0.00209308 -0.00158982 -0.00087646  0.68685672]
 Conferindo: (0.02285960836960972+0j)
Iteração: 1851
Mínimo: [0.00501482 0.00354501 0.00232222 0.00112588 0.68685672]
 Conferindo: (0.02285955392182467+0j)
Iteração: 1852
Mínimo: [0.00501

Mínimo: [0.00331124 0.00218426 0.00151016 0.00074536 0.68685672]
 Conferindo: (0.022859541986188864+0j)
Iteração: 1918
Mínimo: [-0.0033694  -0.00318246 -0.00224519 -0.0011887   0.68685672]
 Conferindo: (0.022859505212331523+0j)
Iteração: 1919
Mínimo: [-0.0033694  -0.00318246 -0.00224519 -0.0011887   0.68685672]
 Conferindo: (0.022859503687958842+0j)
Iteração: 1920
Mínimo: [-0.0033694  -0.00318246 -0.00224519 -0.0011887   0.68685672]
 Conferindo: (0.022859513238923036+0j)
Iteração: 1921
Mínimo: [-0.0033694  -0.00318246 -0.00224519 -0.0011887   0.68685672]
 Conferindo: (0.022859533879275532+0j)
Iteração: 1922
Mínimo: [0.00299755 0.00203959 0.00145203 0.00072362 0.68685672]
 Conferindo: (0.022859499250410007+0j)
Iteração: 1923
Mínimo: [0.00299755 0.00203959 0.00145203 0.00072362 0.68685672]
 Conferindo: (0.022859499614240043+0j)
Iteração: 1924
Mínimo: [0.00299755 0.00203959 0.00145203 0.00072362 0.68685672]
 Conferindo: (0.02285950919859935+0j)
Iteração: 1925
Mínimo: [0.00299755 0.0020395

Mínimo: [0.00477433 0.00349818 0.00234252 0.0011269  0.68685672]
 Conferindo: (0.022859461530908006+0j)
Iteração: 1986
Mínimo: [0.00477433 0.00349818 0.00234252 0.0011269  0.68685672]
 Conferindo: (0.022859461731340208+0j)
Iteração: 1987
Mínimo: [0.00477433 0.00349818 0.00234252 0.0011269  0.68685672]
 Conferindo: (0.022859484985457697+0j)
Iteração: 1988
Mínimo: [-0.00218029 -0.00224964 -0.00174667 -0.00099518  0.68685672]
 Conferindo: (0.022859451335513283+0j)
Iteração: 1989
Mínimo: [-0.00218029 -0.00224964 -0.00174667 -0.00099518  0.68685672]
 Conferindo: (0.022859450339048626+0j)
Iteração: 1990
Mínimo: [-0.00218029 -0.00224964 -0.00174667 -0.00099518  0.68685672]
 Conferindo: (0.022859453966126396+0j)
Iteração: 1991
Mínimo: [-0.00218029 -0.00224964 -0.00174667 -0.00099518  0.68685672]
 Conferindo: (0.02285946222262162+0j)
Iteração: 1992
Mínimo: [-0.00218029 -0.00224964 -0.00174667 -0.00099518  0.68685672]
 Conferindo: (0.022859475109839048+0j)
Iteração: 1993
Mínimo: [-0.00218029 -0.

In [6]:
## Constantes

horizonte = 6
fatorRungeKutta = 0.01

## Chute inicial

x = []
x.append(random.uniform(-1, 1))
for l in range(1, horizonte):
    x.append(x[0])

# Função de origem e destino

ondaOrigem = [[complex(0.80,0.0)],[complex(0.60,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

# Análise das respostas

minimos6 = np.zeros((1,2000), dtype=np.complex_) # Verificar se o mínimo será zero
constantes6 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1
valoresOnda6 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar as componentes de cada onda
valoresFuncaoObjetivo6 = np.zeros((1,2000), dtype=np.complex_) # Conferir se o mínimo está realmente ajustado

# Hamiltoniano

hamiltoniano = np.zeros((2,2), dtype=np.complex_) 
hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]

iteracao = 0
tempoAnalise = 2000

while iteracao < tempoAnalise:
    
    print(f"Iteração: {iteracao+1}")
    
    # Variação do Psi
    valoresOnda6[0][iteracao] = ondaOrigem[0][0]
    valoresOnda6[1][iteracao] = ondaOrigem[1][0]
    
    # Obtenção das constantes que multiplicam Psi
    constantes6[0][iteracao] = ondaOrigem[0][0] / np.exp(-1*complex(0,1)*(1/2*np.pi)*iteracao/100)
    constantes6[1][iteracao] = ondaOrigem[1][0] / np.exp(-3*complex(0,1)*(1/2*np.pi)*iteracao/100)
        
    ## Minimização
    
    resultado = optimize.minimize(funcaoObjetivo, x, args=(iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise), method='CG')

    print(f"Mínimo: {resultado.x}")
    
    # Função objetivo
    
    minimos6[0][iteracao] = resultado.x[0]
    
    valoresFuncaoObjetivo6[0][iteracao] = funcaoObjetivo(resultado.x, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    
    print(f" Conferindo: {valoresFuncaoObjetivo6[0][iteracao]}")
    
    ## Hamiltoniano corntrolado
    
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, resultado.x[0]],[resultado.x[0], 0]]
    hamiltoniano = somar(hamiltoniano, controle)
    
    ## Adaptação da onda ao controle
    
    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)
    
    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]
    
    # Reset
    hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]
    
    x = []
    for l in range(0, horizonte):
        x.append(resultado.x[l])
    
    iteracao += 1

Iteração: 1
Mínimo: [-1.53619732  0.85365297  0.9414809   0.86816883  0.83373497 -0.74525317]
 Conferindo: (0.11979191926957167+0j)
Iteração: 2
Mínimo: [ 0.41678873  0.93781788  0.80507198  0.91685514  0.94630837 -0.74525317]
 Conferindo: (0.11922141145250552+0j)
Iteração: 3
Mínimo: [ 0.45840758  0.93812224  0.82208019  0.88550122  0.90085454 -0.74525317]
 Conferindo: (0.11817415567567517+0j)
Iteração: 4
Mínimo: [ 0.45508378  0.93410167  0.82119033  0.8839203   0.89919105 -0.74525317]
 Conferindo: (0.11711992857405791+0j)
Iteração: 5
Mínimo: [ 0.44991928  0.92839093  0.81875734  0.88115132  0.89685397 -0.74525317]
 Conferindo: (0.11607747063400736+0j)
Iteração: 6
Mínimo: [ 0.4455754   0.92332265  0.81666099  0.87851158  0.89452292 -0.74525317]
 Conferindo: (0.11504584935437764+0j)
Iteração: 7
Mínimo: [ 0.44231417  0.91904833  0.81499147  0.87605573  0.89222522 -0.74525317]
 Conferindo: (0.11402344059738835+0j)
Iteração: 8
Mínimo: [ 0.43986689  0.9152869   0.81350963  0.87361639  0.8898

Mínimo: [ 0.33662353  0.68990538  0.63595899  0.65521975  0.69264018 -0.74525317]
 Conferindo: (0.06950633336762711+0j)
Iteração: 64
Mínimo: [ 0.33480474  0.68612267  0.63265138  0.65145482  0.68905659 -0.74525317]
 Conferindo: (0.06892140880691684+0j)
Iteração: 65
Mínimo: [ 0.33299008  0.68235458  0.6293514   0.64770546  0.68548126 -0.74525317]
 Conferindo: (0.06834279751572572+0j)
Iteração: 66
Mínimo: [ 0.33118178  0.67860204  0.62605936  0.6439723   0.68191515 -0.74525317]
 Conferindo: (0.06777044461338608+0j)
Iteração: 67
Mínimo: [ 0.32937919  0.67486379  0.62277493  0.64025501  0.67835824 -0.74525317]
 Conferindo: (0.06720429478135419+0j)
Iteração: 68
Mínimo: [ 0.32758136  0.67114174  0.61949972  0.63655361  0.67481086 -0.74525317]
 Conferindo: (0.06664429298798362+0j)
Iteração: 69
Mínimo: [ 0.32579338  0.66743621  0.61623435  0.63286967  0.67127364 -0.74525317]
 Conferindo: (0.06609038418401002+0j)
Iteração: 70
Mínimo: [ 0.32400762  0.66374528  0.61297534  0.62920034  0.66774532 

Mínimo: [ 0.22580499  0.46754159  0.44097515  0.44267405  0.47662986 -0.74525317]
 Conferindo: (0.04170322516241563+0j)
Iteração: 132
Mínimo: [ 0.22580499  0.46754159  0.44097515  0.44267405  0.47662986 -0.74525317]
 Conferindo: (0.041432483102900756+0j)
Iteração: 133
Mínimo: [ 0.22400716  0.46369632  0.4368593   0.43850929  0.47299673 -0.74525317]
 Conferindo: (0.04116418054418486+0j)
Iteração: 134
Mínimo: [ 0.22400716  0.46369632  0.4368593   0.43850929  0.47299673 -0.74525317]
 Conferindo: (0.040898774362256106+0j)
Iteração: 135
Mínimo: [ 0.22177615  0.45932704  0.43222881  0.43391644  0.46909557 -0.74525317]
 Conferindo: (0.040635975352519726+0j)
Iteração: 136
Mínimo: [ 0.23630894  0.45510713  0.42220201  0.4177496   0.4491742  -0.74525317]
 Conferindo: (0.040375977707519516+0j)
Iteração: 137
Mínimo: [ 0.23069633  0.44928206  0.41781145  0.41445152  0.44655972 -0.74525317]
 Conferindo: (0.04011948998876951+0j)
Iteração: 138
Mínimo: [ 0.22439028  0.44317546  0.41333261  0.4111328   

Mínimo: [ 0.15091164  0.31873396  0.2894206   0.2901492   0.33574817 -0.74525317]
 Conferindo: (0.029165071610618938+0j)
Iteração: 199
Mínimo: [ 0.15091164  0.31873396  0.2894206   0.2901492   0.33574817 -0.74525317]
 Conferindo: (0.029044504923460886+0j)
Iteração: 200
Mínimo: [ 0.14901902  0.31496015  0.28558501  0.28626439  0.33205589 -0.74525317]
 Conferindo: (0.02892529436290458+0j)
Iteração: 201
Mínimo: [ 0.14901902  0.31496015  0.28558501  0.28626439  0.33205589 -0.74525317]
 Conferindo: (0.02880772475833188+0j)
Iteração: 202
Mínimo: [ 0.14714275  0.31122357  0.28179818  0.28242667  0.32839085 -0.74525317]
 Conferindo: (0.028691478829860777+0j)
Iteração: 203
Mínimo: [ 0.14714275  0.31122357  0.28179818  0.28242667  0.32839085 -0.74525317]
 Conferindo: (0.02857683497956425+0j)
Iteração: 204
Mínimo: [ 0.14529213  0.30752971  0.27806343  0.27863888  0.32475478 -0.74525317]
 Conferindo: (0.028463482205867724+0j)
Iteração: 205
Mínimo: [ 0.14529213  0.30752971  0.27806343  0.27863888  

Mínimo: [ 0.09941211  0.21723676  0.19495359  0.19370908  0.23197975 -0.74525317]
 Conferindo: (0.02395158050609482+0j)
Iteração: 262
Mínimo: [ 0.09941211  0.21723676  0.19495359  0.19370908  0.23197975 -0.74525317]
 Conferindo: (0.023896984774941024+0j)
Iteração: 263
Mínimo: [ 0.09941211  0.21723676  0.19495359  0.19370908  0.23197975 -0.74525317]
 Conferindo: (0.02384282893348347+0j)
Iteração: 264
Mínimo: [ 0.09941211  0.21723676  0.19495359  0.19370908  0.23197975 -0.74525317]
 Conferindo: (0.023789186966815354+0j)
Iteração: 265
Mínimo: [ 0.09760944  0.21367469  0.1912394   0.19006492  0.22866937 -0.74525317]
 Conferindo: (0.023736027372585164+0j)
Iteração: 266
Mínimo: [ 0.09760944  0.21367469  0.1912394   0.19006492  0.22866937 -0.74525317]
 Conferindo: (0.023683570245786906+0j)
Iteração: 267
Mínimo: [ 0.09760944  0.21367469  0.1912394   0.19006492  0.22866937 -0.74525317]
 Conferindo: (0.02363162282649576+0j)
Iteração: 268
Mínimo: [ 0.09760944  0.21367469  0.1912394   0.19006492  

...

Mínimo: [ 1.59713897e-03  1.52924981e-03  8.66500247e-04 -6.63315916e-05
  9.24992446e-04 -7.45253168e-01]
 Conferindo: (0.01963256956539411+0j)
Iteração: 1775
Mínimo: [ 1.59713897e-03  1.52924981e-03  8.66500247e-04 -6.63315916e-05
  9.24992446e-04 -7.45253168e-01]
 Conferindo: (0.019632565443716936+0j)
Iteração: 1776
Mínimo: [ 1.59713897e-03  1.52924981e-03  8.66500247e-04 -6.63315916e-05
  9.24992446e-04 -7.45253168e-01]
 Conferindo: (0.019632564514790944+0j)
Iteração: 1777
Mínimo: [ 1.59713897e-03  1.52924981e-03  8.66500247e-04 -6.63315916e-05
  9.24992446e-04 -7.45253168e-01]
 Conferindo: (0.019632566784366794+0j)
Iteração: 1778
Mínimo: [ 1.59713897e-03  1.52924981e-03  8.66500247e-04 -6.63315916e-05
  9.24992446e-04 -7.45253168e-01]
 Conferindo: (0.019632572255038443+0j)
Iteração: 1779
Mínimo: [ 1.59713897e-03  1.52924981e-03  8.66500247e-04 -6.63315916e-05
  9.24992446e-04 -7.45253168e-01]
 Conferindo: (0.01963258092623905+0j)
Iteração: 1780
Mínimo: [ 1.59713897e-03  1.52924981

Mínimo: [-1.98279283e-03 -1.35644992e-03 -1.21174980e-03 -1.35769532e-03
  1.31992694e-04 -7.45253168e-01]
 Conferindo: (0.019632530173598936+0j)
Iteração: 1830
Mínimo: [-1.98279283e-03 -1.35644992e-03 -1.21174980e-03 -1.35769532e-03
  1.31992694e-04 -7.45253168e-01]
 Conferindo: (0.019632538735010092+0j)
Iteração: 1831
Mínimo: [-1.98279283e-03 -1.35644992e-03 -1.21174980e-03 -1.35769532e-03
  1.31992694e-04 -7.45253168e-01]
 Conferindo: (0.01963255198836878+0j)
Iteração: 1832
Mínimo: [-1.98279283e-03 -1.35644992e-03 -1.21174980e-03 -1.35769532e-03
  1.31992694e-04 -7.45253168e-01]
 Conferindo: (0.019632569928398288+0j)
Iteração: 1833
Mínimo: [ 1.94211133e-03  1.93805595e-03  1.36065628e-03  4.11786601e-04
  1.00954839e-03 -7.45253168e-01]
 Conferindo: (0.019632529838954926+0j)
Iteração: 1834
Mínimo: [ 1.94211133e-03  1.93805595e-03  1.36065628e-03  4.11786601e-04
  1.00954839e-03 -7.45253168e-01]
 Conferindo: (0.019632523832745503+0j)
Iteração: 1835
Mínimo: [ 1.94211133e-03  1.9380559

Mínimo: [-1.58907179e-03 -1.25418884e-03 -1.18237356e-03 -1.30958180e-03
 -5.12290191e-06 -7.45253168e-01]
 Conferindo: (0.019632493682988902+0j)
Iteração: 1884
Mínimo: [-1.58907179e-03 -1.25418884e-03 -1.18237356e-03 -1.30958180e-03
 -5.12290191e-06 -7.45253168e-01]
 Conferindo: (0.019632488643121518+0j)
Iteração: 1885
Mínimo: [-1.58907179e-03 -1.25418884e-03 -1.18237356e-03 -1.30958180e-03
 -5.12290191e-06 -7.45253168e-01]
 Conferindo: (0.019632486552193712+0j)
Iteração: 1886
Mínimo: [-1.58907179e-03 -1.25418884e-03 -1.18237356e-03 -1.30958180e-03
 -5.12290191e-06 -7.45253168e-01]
 Conferindo: (0.019632487417053375+0j)
Iteração: 1887
Mínimo: [-1.58907179e-03 -1.25418884e-03 -1.18237356e-03 -1.30958180e-03
 -5.12290191e-06 -7.45253168e-01]
 Conferindo: (0.01963249124162978+0j)
Iteração: 1888
Mínimo: [-1.58907179e-03 -1.25418884e-03 -1.18237356e-03 -1.30958180e-03
 -5.12290191e-06 -7.45253168e-01]
 Conferindo: (0.01963249802693035+0j)
Iteração: 1889
Mínimo: [-1.58907179e-03 -1.25418884

Mínimo: [ 2.22682269e-03  1.88406718e-03  1.29467883e-03  4.45370791e-04
  7.77873049e-04 -7.45253168e-01]
 Conferindo: (0.019632454610649316+0j)
Iteração: 1937
Mínimo: [ 2.22682269e-03  1.88406718e-03  1.29467883e-03  4.45370791e-04
  7.77873049e-04 -7.45253168e-01]
 Conferindo: (0.019632454617111193+0j)
Iteração: 1938
Mínimo: [ 2.22682269e-03  1.88406718e-03  1.29467883e-03  4.45370791e-04
  7.77873049e-04 -7.45253168e-01]
 Conferindo: (0.01963246073303331+0j)
Iteração: 1939
Mínimo: [ 2.22682269e-03  1.88406718e-03  1.29467883e-03  4.45370791e-04
  7.77873049e-04 -7.45253168e-01]
 Conferindo: (0.019632472962392553+0j)
Iteração: 1940
Mínimo: [ 2.22682269e-03  1.88406718e-03  1.29467883e-03  4.45370791e-04
  7.77873049e-04 -7.45253168e-01]
 Conferindo: (0.019632491303131463+0j)
Iteração: 1941
Mínimo: [-1.61654254e-03 -1.38574677e-03 -1.26311853e-03 -1.30378595e-03
 -1.25565495e-04 -7.45253168e-01]
 Conferindo: (0.0196324546179706+0j)
Iteração: 1942
Mínimo: [-1.61654254e-03 -1.38574677e

Mínimo: [ 2.25881898e-03  1.84563988e-03  1.29833101e-03  5.09522153e-04
  7.08322810e-04 -7.45253168e-01]
 Conferindo: (0.01963242092076188+0j)
Iteração: 1994
Mínimo: [ 2.25881898e-03  1.84563988e-03  1.29833101e-03  5.09522153e-04
  7.08322810e-04 -7.45253168e-01]
 Conferindo: (0.01963241471202784+0j)
Iteração: 1995
Mínimo: [ 2.25881898e-03  1.84563988e-03  1.29833101e-03  5.09522153e-04
  7.08322810e-04 -7.45253168e-01]
 Conferindo: (0.019632414779046512+0j)
Iteração: 1996
Mínimo: [ 2.25881898e-03  1.84563988e-03  1.29833101e-03  5.09522153e-04
  7.08322810e-04 -7.45253168e-01]
 Conferindo: (0.019632421132093122+0j)
Iteração: 1997
Mínimo: [ 2.25881898e-03  1.84563988e-03  1.29833101e-03  5.09522153e-04
  7.08322810e-04 -7.45253168e-01]
 Conferindo: (0.019632433775238264+0j)
Iteração: 1998
Mínimo: [ 2.25881898e-03  1.84563988e-03  1.29833101e-03  5.09522153e-04
  7.08322810e-04 -7.45253168e-01]
 Conferindo: (0.019632452706347087+0j)
Iteração: 1999
Mínimo: [-1.64202842e-03 -1.46895299

In [7]:
## Constantes

horizonte = 7
fatorRungeKutta = 0.01

## Chute inicial

x = []
x.append(random.uniform(-1, 1))
for l in range(1, horizonte):
    x.append(x[0])

# Função de origem e destino

ondaOrigem = [[complex(0.80,0.0)],[complex(0.60,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

# Análise das respostas

minimos7 = np.zeros((1,2000), dtype=np.complex_) # Verificar se o mínimo será zero
constantes7 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1
valoresOnda7 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar as componentes de cada onda
valoresFuncaoObjetivo7 = np.zeros((1,2000), dtype=np.complex_) # Conferir se o mínimo está realmente ajustado

# Hamiltoniano

hamiltoniano = np.zeros((2,2), dtype=np.complex_) 
hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]

iteracao = 0
tempoAnalise = 2000

while iteracao < tempoAnalise:
    
    print(f"Iteração: {iteracao+1}")
    
    # Variação do Psi
    valoresOnda7[0][iteracao] = ondaOrigem[0][0]
    valoresOnda7[1][iteracao] = ondaOrigem[1][0]
    
    # Obtenção das constantes que multiplicam Psi
    constantes7[0][iteracao] = ondaOrigem[0][0] / np.exp(-1*complex(0,1)*(1/2*np.pi)*iteracao/100)
    constantes7[1][iteracao] = ondaOrigem[1][0] / np.exp(-3*complex(0,1)*(1/2*np.pi)*iteracao/100)
        
    ## Minimização
    
    resultado = optimize.minimize(funcaoObjetivo, x, args=(iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise), method='CG')

    print(f"Mínimo: {resultado.x}")
    
    # Função objetivo
    
    minimos7[0][iteracao] = resultado.x[0]
    
    valoresFuncaoObjetivo7[0][iteracao] = funcaoObjetivo(resultado.x, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    
    print(f" Conferindo: {valoresFuncaoObjetivo7[0][iteracao]}")
    
    ## Hamiltoniano corntrolado
    
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, resultado.x[0]],[resultado.x[0], 0]]
    hamiltoniano = somar(hamiltoniano, controle)
    
    ## Adaptação da onda ao controle
    
    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)
    
    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]
    
    # Reset
    hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]
    
    x = []
    for l in range(0, horizonte):
        x.append(resultado.x[l])
    
    iteracao += 1

Iteração: 1
Mínimo: [-1.96502311  0.84473981  0.92670986  0.85428803  0.89433436  0.90645518
  0.42396042]
 Conferindo: (0.13930166624620968+0j)
Iteração: 2
Mínimo: [0.43979926 0.91576944 0.82335908 0.90579777 0.89317802 0.86100754
 0.42396042]
 Conferindo: (0.13850310118371384+0j)
Iteração: 3
Mínimo: [0.43979926 0.91576944 0.82335908 0.90579777 0.89317802 0.86100754
 0.42396042]
 Conferindo: (0.13702798866040666+0j)
Iteração: 4
Mínimo: [0.43979926 0.91576944 0.82335908 0.90579777 0.89317802 0.86100754
 0.42396042]
 Conferindo: (0.13555720196659604+0j)
Iteração: 5
Mínimo: [0.43979926 0.91576944 0.82335908 0.90579777 0.89317802 0.86100754
 0.42396042]
 Conferindo: (0.1340922319906129+0j)
Iteração: 6
Mínimo: [0.46975545 0.91669695 0.82260519 0.87811675 0.8625274  0.8452633
 0.42396042]
 Conferindo: (0.13263402244138991+0j)
Iteração: 7
Mínimo: [0.45768887 0.90479696 0.81402226 0.87063524 0.85739357 0.84293233
 0.42396042]
 Conferindo: (0.1311871218515987+0j)
Iteração: 8
Mínimo: [0.4486599

Mínimo: [0.32250964 0.64610599 0.60947066 0.6157551  0.60945832 0.6421614
 0.42396042]
 Conferindo: (0.07374504615656695+0j)
Iteração: 62
Mínimo: [0.31918244 0.64258103 0.60698603 0.61360176 0.60755429 0.64025877
 0.42396042]
 Conferindo: (0.07300053257559394+0j)
Iteração: 63
Mínimo: [0.31657275 0.63943494 0.60460765 0.61134268 0.60543842 0.6381537
 0.42396042]
 Conferindo: (0.07226403593675752+0j)
Iteração: 64
Mínimo: [0.31657275 0.63943494 0.60460765 0.61134268 0.60543842 0.6381537
 0.42396042]
 Conferindo: (0.07153501495586517+0j)
Iteração: 65
Mínimo: [0.31142306 0.63339478 0.59923828 0.60633973 0.60116031 0.63481554
 0.42396042]
 Conferindo: (0.07081313922691568+0j)
Iteração: 66
Mínimo: [0.31142306 0.63339478 0.59923828 0.60633973 0.60116031 0.63481554
 0.42396042]
 Conferindo: (0.07009910705267211+0j)
Iteração: 67
Mínimo: [0.30772454 0.62802469 0.59398397 0.60102722 0.596355   0.63097659
 0.42396042]
 Conferindo: (0.06939160246438716+0j)
Iteração: 68
Mínimo: [0.30772454 0.62802469

Mínimo: [0.21642635 0.44417909 0.41964077 0.41957411 0.41568955 0.45216408
 0.42396042]
 Conferindo: (0.04185624964942714+0j)
Iteração: 123
Mínimo: [0.21437335 0.4401528  0.41525466 0.4150192  0.41139347 0.44849935
 0.42396042]
 Conferindo: (0.04151295520630428+0j)
Iteração: 124
Mínimo: [0.21437335 0.4401528  0.41525466 0.4150192  0.41139347 0.44849935
 0.42396042]
 Conferindo: (0.04117368147419333+0j)
Iteração: 125
Mínimo: [0.23528393 0.43968077 0.40722013 0.3995723  0.39067498 0.42445028
 0.42396042]
 Conferindo: (0.04083788982292612+0j)
Iteração: 126
Mínimo: [0.22421878 0.42928337 0.39911731 0.39369799 0.38671514 0.42174004
 0.42396042]
 Conferindo: (0.040507275786757134+0j)
Iteração: 127
Mínimo: [0.21650928 0.42237285 0.39420939 0.39040392 0.3845243  0.41989961
 0.42396042]
 Conferindo: (0.04018678757259772+0j)
Iteração: 128
Mínimo: [0.21079842 0.41741948 0.3909873  0.38838184 0.38314454 0.41842415
 0.42396042]
 Conferindo: (0.03987360320862629+0j)
Iteração: 129
Mínimo: [0.20665698

Mínimo: [0.14250927 0.29816287 0.27713801 0.27665836 0.2741499  0.31006557
 0.42396042]
 Conferindo: (0.027847707040456512+0j)
Iteração: 183
Mínimo: [0.14250927 0.29816287 0.27713801 0.27665836 0.2741499  0.31006557
 0.42396042]
 Conferindo: (0.027695351050259876+0j)
Iteração: 184
Mínimo: [0.14221392 0.295783   0.27413726 0.27324722 0.27068232 0.30687626
 0.42396042]
 Conferindo: (0.027544603063202375+0j)
Iteração: 185
Mínimo: [0.14221392 0.295783   0.27413726 0.27324722 0.27068232 0.30687626
 0.42396042]
 Conferindo: (0.027395754998081275+0j)
Iteração: 186
Mínimo: [0.14041395 0.2919937  0.26992582 0.26888414 0.2665513  0.30333531
 0.42396042]
 Conferindo: (0.027248780506359097+0j)
Iteração: 187
Mínimo: [0.14041395 0.2919937  0.26992582 0.26888414 0.2665513  0.30333531
 0.42396042]
 Conferindo: (0.027103966815910624+0j)
Iteração: 188
Mínimo: [0.13873482 0.28834414 0.26587809 0.26468254 0.2625401  0.29984732
 0.42396042]
 Conferindo: (0.026961022252728374+0j)
Iteração: 189
Mínimo: [0.13

Mínimo: [0.09431688 0.20571607 0.17945547 0.17748385 0.17839729 0.22436631
 0.42396042]
 Conferindo: (0.021668559150372538+0j)
Iteração: 242
Mínimo: [0.09431688 0.20571607 0.17945547 0.17748385 0.17839729 0.22436631
 0.42396042]
 Conferindo: (0.021601752321566903+0j)
Iteração: 243
Mínimo: [0.09431688 0.20571607 0.17945547 0.17748385 0.17839729 0.22436631
 0.42396042]
 Conferindo: (0.02153584445484828+0j)
Iteração: 244
Mínimo: [0.09228158 0.20175782 0.17534518 0.1733314  0.17429906 0.22057939
 0.42396042]
 Conferindo: (0.021470786913737355+0j)
Iteração: 245
Mínimo: [0.09228158 0.20175782 0.17534518 0.1733314  0.17429906 0.22057939
 0.42396042]
 Conferindo: (0.021406813711178568+0j)
Iteração: 246
Mínimo: [0.09228158 0.20175782 0.17534518 0.1733314  0.17429906 0.22057939
 0.42396042]
 Conferindo: (0.021343702905015326+0j)
Iteração: 247
Mínimo: [0.09028583 0.19786658 0.17132389 0.16927359 0.17027825 0.2168317
 0.42396042]
 Conferindo: (0.021281406993913305+0j)
Iteração: 248
Mínimo: [0.0902

...

Mínimo: [-1.50114455e-03 -9.67066840e-04 -1.69177793e-03 -1.74248324e-04
 -1.67474912e-03  4.77042751e-04  4.23960421e-01]
 Conferindo: (0.01703576012265275+0j)
Iteração: 1760
Mínimo: [-1.50114455e-03 -9.67066840e-04 -1.69177793e-03 -1.74248324e-04
 -1.67474912e-03  4.77042751e-04  4.23960421e-01]
 Conferindo: (0.017035758583529768+0j)
Iteração: 1761
Mínimo: [-1.50114455e-03 -9.67066840e-04 -1.69177793e-03 -1.74248324e-04
 -1.67474912e-03  4.77042751e-04  4.23960421e-01]
 Conferindo: (0.017035760135480242+0j)
Iteração: 1762
Mínimo: [-1.50114455e-03 -9.67066840e-04 -1.69177793e-03 -1.74248324e-04
 -1.67474912e-03  4.77042751e-04  4.23960421e-01]
 Conferindo: (0.017035764781219702+0j)
Iteração: 1763
Mínimo: [-1.50114455e-03 -9.67066840e-04 -1.69177793e-03 -1.74248324e-04
 -1.67474912e-03  4.77042751e-04  4.23960421e-01]
 Conferindo: (0.017035772520412197+0j)
Iteração: 1764
Mínimo: [-1.50114455e-03 -9.67066840e-04 -1.69177793e-03 -1.74248324e-04
 -1.67474912e-03  4.77042751e-04  4.2396042

Mínimo: [ 1.61123395e-03  1.69654667e-03  6.17883974e-04  1.51400244e-03
 -3.64351105e-04  9.77114263e-04  4.23960421e-01]
 Conferindo: (0.017035762238337967+0j)
Iteração: 1808
Mínimo: [-1.66829473e-03 -1.20099844e-03 -1.79349402e-03 -3.69357506e-04
 -1.62697921e-03  3.16784479e-04  4.23960421e-01]
 Conferindo: (0.01703572767213492+0j)
Iteração: 1809
Mínimo: [-1.66829473e-03 -1.20099844e-03 -1.79349402e-03 -3.69357506e-04
 -1.62697921e-03  3.16784479e-04  4.23960421e-01]
 Conferindo: (0.017035725977097245+0j)
Iteração: 1810
Mínimo: [-1.66829473e-03 -1.20099844e-03 -1.79349402e-03 -3.69357506e-04
 -1.62697921e-03  3.16784479e-04  4.23960421e-01]
 Conferindo: (0.017035728107659198+0j)
Iteração: 1811
Mínimo: [-1.66829473e-03 -1.20099844e-03 -1.79349402e-03 -3.69357506e-04
 -1.62697921e-03  3.16784479e-04  4.23960421e-01]
 Conferindo: (0.017035734067090687+0j)
Iteração: 1812
Mínimo: [-1.66829473e-03 -1.20099844e-03 -1.79349402e-03 -3.69357506e-04
 -1.62697921e-03  3.16784479e-04  4.2396042

Mínimo: [-1.53958583e-03 -1.11756018e-03 -1.61942089e-03 -3.31539027e-04
 -1.43720714e-03  2.56743052e-04  4.23960421e-01]
 Conferindo: (0.01703569409884731+0j)
Iteração: 1857
Mínimo: [-1.53958583e-03 -1.11756018e-03 -1.61942089e-03 -3.31539027e-04
 -1.43720714e-03  2.56743052e-04  4.23960421e-01]
 Conferindo: (0.017035692495863772+0j)
Iteração: 1858
Mínimo: [-1.53958583e-03 -1.11756018e-03 -1.61942089e-03 -3.31539027e-04
 -1.43720714e-03  2.56743052e-04  4.23960421e-01]
 Conferindo: (0.01703569415860914+0j)
Iteração: 1859
Mínimo: [-1.53958583e-03 -1.11756018e-03 -1.61942089e-03 -3.31539027e-04
 -1.43720714e-03  2.56743052e-04  4.23960421e-01]
 Conferindo: (0.01703569908990832+0j)
Iteração: 1860
Mínimo: [-1.53958583e-03 -1.11756018e-03 -1.61942089e-03 -3.31539027e-04
 -1.43720714e-03  2.56743052e-04  4.23960421e-01]
 Conferindo: (0.017035707289360373+0j)
Iteração: 1861
Mínimo: [-1.53958583e-03 -1.11756018e-03 -1.61942089e-03 -3.31539027e-04
 -1.43720714e-03  2.56743052e-04  4.23960421e

Mínimo: [-1.19990424e-03 -9.83569123e-04 -1.49824074e-03 -3.72022368e-04
 -1.34173578e-03  1.63516530e-04  4.23960421e-01]
 Conferindo: (0.017035660816833584+0j)
Iteração: 1906
Mínimo: [-1.19990424e-03 -9.83569123e-04 -1.49824074e-03 -3.72022368e-04
 -1.34173578e-03  1.63516530e-04  4.23960421e-01]
 Conferindo: (0.01703565930364487+0j)
Iteração: 1907
Mínimo: [-1.19990424e-03 -9.83569123e-04 -1.49824074e-03 -3.72022368e-04
 -1.34173578e-03  1.63516530e-04  4.23960421e-01]
 Conferindo: (0.017035659735265066+0j)
Iteração: 1908
Mínimo: [-1.19990424e-03 -9.83569123e-04 -1.49824074e-03 -3.72022368e-04
 -1.34173578e-03  1.63516530e-04  4.23960421e-01]
 Conferindo: (0.017035662113738546+0j)
Iteração: 1909
Mínimo: [-1.19990424e-03 -9.83569123e-04 -1.49824074e-03 -3.72022368e-04
 -1.34173578e-03  1.63516530e-04  4.23960421e-01]
 Conferindo: (0.01703566643918732+0j)
Iteração: 1910
Mínimo: [-1.19990424e-03 -9.83569123e-04 -1.49824074e-03 -3.72022368e-04
 -1.34173578e-03  1.63516530e-04  4.23960421

Mínimo: [-1.73247532e-03 -1.46974945e-03 -1.79942620e-03 -6.64318417e-04
 -1.40313330e-03  1.75370027e-05  4.23960421e-01]
 Conferindo: (0.017035628298033782+0j)
Iteração: 1957
Mínimo: [-1.73247532e-03 -1.46974945e-03 -1.79942620e-03 -6.64318417e-04
 -1.40313330e-03  1.75370027e-05  4.23960421e-01]
 Conferindo: (0.017035626220295308+0j)
Iteração: 1958
Mínimo: [-1.73247532e-03 -1.46974945e-03 -1.79942620e-03 -6.64318417e-04
 -1.40313330e-03  1.75370027e-05  4.23960421e-01]
 Conferindo: (0.01703562825626853+0j)
Iteração: 1959
Mínimo: [-1.73247532e-03 -1.46974945e-03 -1.79942620e-03 -6.64318417e-04
 -1.40313330e-03  1.75370027e-05  4.23960421e-01]
 Conferindo: (0.017035634409817275+0j)
Iteração: 1960
Mínimo: [-1.73247532e-03 -1.46974945e-03 -1.79942620e-03 -6.64318417e-04
 -1.40313330e-03  1.75370027e-05  4.23960421e-01]
 Conferindo: (0.017035644680744113+0j)
Iteração: 1961
Mínimo: [-1.73247532e-03 -1.46974945e-03 -1.79942620e-03 -6.64318417e-04
 -1.40313330e-03  1.75370027e-05  4.2396042

In [8]:
## Constantes

horizonte = 8
fatorRungeKutta = 0.01

## Chute inicial

x = []
x.append(random.uniform(-1, 1))
for l in range(1, horizonte):
    x.append(x[0])

# Função de origem e destino

ondaOrigem = [[complex(0.80,0.0)],[complex(0.60,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

# Análise das respostas

minimos8 = np.zeros((1,2000), dtype=np.complex_) # Verificar se o mínimo será zero
constantes8 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1
valoresOnda8 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar as componentes de cada onda
valoresFuncaoObjetivo8 = np.zeros((1,2000), dtype=np.complex_) # Conferir se o mínimo está realmente ajustado

# Hamiltoniano

hamiltoniano = np.zeros((2,2), dtype=np.complex_) 
hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]

iteracao = 0
tempoAnalise = 2000

while iteracao < tempoAnalise:
    
    print(f"Iteração: {iteracao+1}")
    
    # Variação do Psi
    valoresOnda8[0][iteracao] = ondaOrigem[0][0]
    valoresOnda8[1][iteracao] = ondaOrigem[1][0]
    
    # Obtenção das constantes que multiplicam Psi
    constantes8[0][iteracao] = ondaOrigem[0][0] / np.exp(-1*complex(0,1)*(1/2*np.pi)*iteracao/100)
    constantes8[1][iteracao] = ondaOrigem[1][0] / np.exp(-3*complex(0,1)*(1/2*np.pi)*iteracao/100)
        
    ## Minimização
    
    resultado = optimize.minimize(funcaoObjetivo, x, args=(iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise), method='CG')

    print(f"Mínimo: {resultado.x}")
    
    # Função objetivo
    
    minimos8[0][iteracao] = resultado.x[0]
    
    valoresFuncaoObjetivo8[0][iteracao] = funcaoObjetivo(resultado.x, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    
    print(f" Conferindo: {valoresFuncaoObjetivo8[0][iteracao]}")
    
    ## Hamiltoniano corntrolado
    
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, resultado.x[0]],[resultado.x[0], 0]]
    hamiltoniano = somar(hamiltoniano, controle)
    
    ## Adaptação da onda ao controle
    
    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)
    
    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]
    
    # Reset
    hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]
    
    x = []
    for l in range(0, horizonte):
        x.append(resultado.x[l])
    
    iteracao += 1

Iteração: 1
Mínimo: [-2.41095088  0.88502715  0.93283646  0.8405084   0.85917164  0.88362593
  0.88113615  0.62622663]
 Conferindo: (0.1585837559910356+0j)
Iteração: 2
Mínimo: [0.40783647 0.98006268 0.82646768 0.89047997 0.88402537 0.85203335
 0.84846034 0.62622663]
 Conferindo: (0.15752223277575825+0j)
Iteração: 3
Mínimo: [0.46855402 0.97088747 0.82985435 0.87518306 0.86850861 0.85102345
 0.8534552  0.62622663]
 Conferindo: (0.15553357181281532+0j)
Iteração: 4
Mínimo: [0.45996154 0.96088073 0.82279099 0.86929588 0.86420817 0.84854785
 0.8523539  0.62622663]
 Conferindo: (0.15353645995406962+0j)
Iteração: 5
Mínimo: [0.45344429 0.95278786 0.81727636 0.86452196 0.8605714  0.84634712
 0.85128486 0.62622663]
 Conferindo: (0.1515693736476243+0j)
Iteração: 6
Mínimo: [0.44832908 0.94586548 0.81257964 0.86021685 0.85708857 0.84407439
 0.85007475 0.62622663]
 Conferindo: (0.14962828309837117+0j)
Iteração: 7
Mínimo: [0.44424535 0.9396721  0.80830149 0.85603122 0.85348877 0.8415533
 0.84863705 0.

Mínimo: [0.33074462 0.6488696  0.59911969 0.60606661 0.59445847 0.5912704
 0.63085083 0.62622663]
 Conferindo: (0.0794361286335078+0j)
Iteração: 57
Mínimo: [0.32389573 0.64157138 0.59304289 0.60089403 0.59036612 0.58809029
 0.6284148  0.62622663]
 Conferindo: (0.07846717513152716+0j)
Iteração: 58
Mínimo: [0.31816183 0.63538374 0.58794557 0.59649159 0.58679735 0.58519594
 0.62607636 0.62622663]
 Conferindo: (0.07751602495376896+0j)
Iteração: 59
Mínimo: [0.31352367 0.63017718 0.58364798 0.59266575 0.58357987 0.58245836
 0.62376178 0.62622663]
 Conferindo: (0.07658030108366184+0j)
Iteração: 60
Mínimo: [0.30979309 0.62569207 0.57987744 0.58916126 0.58050168 0.57972336
 0.62138278 0.62622663]
 Conferindo: (0.07565821652145645+0j)
Iteração: 61
Mínimo: [0.30676925 0.62167797 0.57638812 0.58575366 0.57737768 0.5768523
 0.61885129 0.62622663]
 Conferindo: (0.07474852836076937+0j)
Iteração: 62
Mínimo: [0.30435922 0.61800715 0.57305476 0.58233105 0.57411892 0.57378109
 0.61612612 0.62622663]
 Con

Mínimo: [0.22527841 0.43448225 0.4028426  0.40063802 0.39221072 0.38669311
 0.42405285 0.62622663]
 Conferindo: (0.04268847905514188+0j)
Iteração: 112
Mínimo: [0.21891161 0.42802564 0.39748791 0.39634012 0.38898226 0.38431069
 0.42216512 0.62622663]
 Conferindo: (0.04225718937042753+0j)
Iteração: 113
Mínimo: [0.21365656 0.42274238 0.3932229  0.39294601 0.38641379 0.38233236
 0.42045122 0.62622663]
 Conferindo: (0.04183565468629131+0j)
Iteração: 114
Mínimo: [0.20951277 0.41851794 0.38986498 0.39025202 0.38432118 0.3806225
 0.4188422  0.62622663]
 Conferindo: (0.04142222684200623+0j)
Iteração: 115
Mínimo: [0.20623405 0.41503927 0.38708793 0.38795595 0.38245366 0.37900257
 0.41724087 0.62622663]
 Conferindo: (0.04101566400926823+0j)
Iteração: 116
Mínimo: [0.20365698 0.41209864 0.38468007 0.38586332 0.3806537  0.37736023
 0.4155832  0.62622663]
 Conferindo: (0.04061509672526344+0j)
Iteração: 117
Mínimo: [0.20161924 0.40950228 0.38245584 0.38381024 0.37878933 0.37559789
 0.41380429 0.626226

Mínimo: [0.1391099  0.28497775 0.26072839 0.26026103 0.25675437 0.25390681
 0.29187528 0.62622663]
 Conferindo: (0.026589468755577467+0j)
Iteração: 167
Mínimo: [0.1391099  0.28497775 0.26072839 0.26026103 0.25675437 0.25390681
 0.29187528 0.62622663]
 Conferindo: (0.026406105084517045+0j)
Iteração: 168
Mínimo: [0.13546698 0.28095695 0.25722158 0.25718588 0.25411749 0.251654
 0.28990897 0.62622663]
 Conferindo: (0.026225204961556874+0j)
Iteração: 169
Mínimo: [0.13546698 0.28095695 0.25722158 0.25718588 0.25411749 0.251654
 0.28990897 0.62622663]
 Conferindo: (0.02604699799009628+0j)
Iteração: 170
Mínimo: [0.13297289 0.27776663 0.25427328 0.25441807 0.25158126 0.24935736
 0.28783183 0.62622663]
 Conferindo: (0.02587089024734521+0j)
Iteração: 171
Mínimo: [0.13297289 0.27776663 0.25427328 0.25441807 0.25158126 0.24935736
 0.28783183 0.62622663]
 Conferindo: (0.02569711470525415+0j)
Iteração: 172
Mínimo: [0.13117698 0.27491125 0.25141523 0.25154358 0.2488076  0.2467662
 0.28548559 0.6262266

Mínimo: [0.08816211 0.18958774 0.16949431 0.16996806 0.16840802 0.16627036
 0.20150719 0.62622663]
 Conferindo: (0.01962979610578512+0j)
Iteração: 223
Mínimo: [0.08816211 0.18958774 0.16949431 0.16996806 0.16840802 0.16627036
 0.20150719 0.62622663]
 Conferindo: (0.019551649317717794+0j)
Iteração: 224
Mínimo: [0.08816211 0.18958774 0.16949431 0.16996806 0.16840802 0.16627036
 0.20150719 0.62622663]
 Conferindo: (0.01947438929215494+0j)
Iteração: 225
Mínimo: [0.08672136 0.18684589 0.16651793 0.16683914 0.16532242 0.16341935
 0.19903486 0.62622663]
 Conferindo: (0.01939801180225836+0j)
Iteração: 226
Mínimo: [0.08672136 0.18684589 0.16651793 0.16683914 0.16532242 0.16341935
 0.19903486 0.62622663]
 Conferindo: (0.019322709683316362+0j)
Iteração: 227
Mínimo: [0.08672136 0.18684589 0.16651793 0.16683914 0.16532242 0.16341935
 0.19903486 0.62622663]
 Conferindo: (0.019248352906845505+0j)
Iteração: 228
Mínimo: [0.09746591 0.18670961 0.16280021 0.15960371 0.15534187 0.15144702
 0.18565165 0.62

...

Mínimo: [ 7.52983622e-04  2.02403526e-03 -2.46934624e-04  1.27104439e-03
  1.31834473e-03 -1.73583685e-03  2.30105348e-03  6.26226627e-01]
 Conferindo: (0.014669633765400905+0j)
Iteração: 1790
Mínimo: [ 7.52983622e-04  2.02403526e-03 -2.46934624e-04  1.27104439e-03
  1.31834473e-03 -1.73583685e-03  2.30105348e-03  6.26226627e-01]
 Conferindo: (0.01466963708384833+0j)
Iteração: 1791
Mínimo: [ 7.52983622e-04  2.02403526e-03 -2.46934624e-04  1.27104439e-03
  1.31834473e-03 -1.73583685e-03  2.30105348e-03  6.26226627e-01]
 Conferindo: (0.014669641323092272+0j)
Iteração: 1792
Mínimo: [ 7.52983622e-04  2.02403526e-03 -2.46934624e-04  1.27104439e-03
  1.31834473e-03 -1.73583685e-03  2.30105348e-03  6.26226627e-01]
 Conferindo: (0.014669646479513195+0j)
Iteração: 1793
Mínimo: [ 7.52983622e-04  2.02403526e-03 -2.46934624e-04  1.27104439e-03
  1.31834473e-03 -1.73583685e-03  2.30105348e-03  6.26226627e-01]
 Conferindo: (0.014669652548589882+0j)
Iteração: 1794
Mínimo: [ 7.52983622e-04  2.02403526

Mínimo: [ 7.02411593e-04  1.91346595e-03 -1.77657562e-04  1.22100492e-03
  1.27793477e-03 -1.57828826e-03  2.11399255e-03  6.26226627e-01]
 Conferindo: (0.014669598409835177+0j)
Iteração: 1833
Mínimo: [ 7.02411593e-04  1.91346595e-03 -1.77657562e-04  1.22100492e-03
  1.27793477e-03 -1.57828826e-03  2.11399255e-03  6.26226627e-01]
 Conferindo: (0.01466959712347781+0j)
Iteração: 1834
Mínimo: [ 7.02411593e-04  1.91346595e-03 -1.77657562e-04  1.22100492e-03
  1.27793477e-03 -1.57828826e-03  2.11399255e-03  6.26226627e-01]
 Conferindo: (0.01466959663319887+0j)
Iteração: 1835
Mínimo: [ 7.02411593e-04  1.91346595e-03 -1.77657562e-04  1.22100492e-03
  1.27793477e-03 -1.57828826e-03  2.11399255e-03  6.26226627e-01]
 Conferindo: (0.014669596939900777+0j)
Iteração: 1836
Mínimo: [ 7.02411593e-04  1.91346595e-03 -1.77657562e-04  1.22100492e-03
  1.27793477e-03 -1.57828826e-03  2.11399255e-03  6.26226627e-01]
 Conferindo: (0.014669598043699801+0j)
Iteração: 1837
Mínimo: [ 7.02411593e-04  1.91346595e

Mínimo: [-1.51100602e-03 -1.46086147e-04 -1.84785003e-03 -2.48530616e-04
  1.42265996e-04 -2.20230517e-03  1.59512339e-03  6.26226627e-01]
 Conferindo: (0.01466956855367768+0j)
Iteração: 1877
Mínimo: [-1.51100602e-03 -1.46086147e-04 -1.84785003e-03 -2.48530616e-04
  1.42265996e-04 -2.20230517e-03  1.59512339e-03  6.26226627e-01]
 Conferindo: (0.014669571290978574+0j)
Iteração: 1878
Mínimo: [-1.51100602e-03 -1.46086147e-04 -1.84785003e-03 -2.48530616e-04
  1.42265996e-04 -2.20230517e-03  1.59512339e-03  6.26226627e-01]
 Conferindo: (0.014669577702724899+0j)
Iteração: 1879
Mínimo: [-1.51100602e-03 -1.46086147e-04 -1.84785003e-03 -2.48530616e-04
  1.42265996e-04 -2.20230517e-03  1.59512339e-03  6.26226627e-01]
 Conferindo: (0.014669587786835792+0j)
Iteração: 1880
Mínimo: [ 6.85589046e-04  1.82407190e-03 -1.12719149e-04  1.16909757e-03
  1.23078458e-03 -1.43655586e-03  1.94210355e-03  6.26226627e-01]
 Conferindo: (0.014669566003937708+0j)
Iteração: 1881
Mínimo: [ 6.85589046e-04  1.82407190

Mínimo: [ 7.45156946e-04  1.84992417e-03  1.09552199e-05  1.21302618e-03
  1.26477557e-03 -1.30175132e-03  1.86168797e-03  6.26226627e-01]
 Conferindo: (0.014669563348162622+0j)
Iteração: 1923
Mínimo: [ 7.45156946e-04  1.84992417e-03  1.09552199e-05  1.21302618e-03
  1.26477557e-03 -1.30175132e-03  1.86168797e-03  6.26226627e-01]
 Conferindo: (0.014669569842424658+0j)
Iteração: 1924
Mínimo: [-1.58953715e-03 -2.67400444e-04 -1.80770705e-03 -3.07485508e-04
  9.47673817e-05 -2.06759812e-03  1.43981974e-03  6.26226627e-01]
 Conferindo: (0.014669537031225924+0j)
Iteração: 1925
Mínimo: [-1.58953715e-03 -2.67400444e-04 -1.80770705e-03 -3.07485508e-04
  9.47673817e-05 -2.06759812e-03  1.43981974e-03  6.26226627e-01]
 Conferindo: (0.01466953607616564+0j)
Iteração: 1926
Mínimo: [-1.58953715e-03 -2.67400444e-04 -1.80770705e-03 -3.07485508e-04
  9.47673817e-05 -2.06759812e-03  1.43981974e-03  6.26226627e-01]
 Conferindo: (0.014669539207811534+0j)
Iteração: 1927
Mínimo: [-1.58953715e-03 -2.67400444

 Conferindo: (0.01466951299666166+0j)
Iteração: 1967
Mínimo: [ 7.04983726e-04  1.74870439e-03  4.52407364e-05  1.14568034e-03
  1.20434969e-03 -1.18679859e-03  1.71209374e-03  6.26226627e-01]
 Conferindo: (0.014669515600791808+0j)
Iteração: 1968
Mínimo: [ 7.04983726e-04  1.74870439e-03  4.52407364e-05  1.14568034e-03
  1.20434969e-03 -1.18679859e-03  1.71209374e-03  6.26226627e-01]
 Conferindo: (0.014669518987489326+0j)
Iteração: 1969
Mínimo: [ 7.04983726e-04  1.74870439e-03  4.52407364e-05  1.14568034e-03
  1.20434969e-03 -1.18679859e-03  1.71209374e-03  6.26226627e-01]
 Conferindo: (0.014669523153820238+0j)
Iteração: 1970
Mínimo: [ 7.04983726e-04  1.74870439e-03  4.52407364e-05  1.14568034e-03
  1.20434969e-03 -1.18679859e-03  1.71209374e-03  6.26226627e-01]
 Conferindo: (0.014669528096081493+0j)
Iteração: 1971
Mínimo: [ 7.04983726e-04  1.74870439e-03  4.52407364e-05  1.14568034e-03
  1.20434969e-03 -1.18679859e-03  1.71209374e-03  6.26226627e-01]
 Conferindo: (0.01466953380980571+0j

In [10]:
## Constantes

horizonte = 9
fatorRungeKutta = 0.01

## Chute inicial

x = []
x.append(random.uniform(-1, 1))
for l in range(1, horizonte):
    x.append(x[0])

# Função de origem e destino

ondaOrigem = [[complex(0.80,0.0)],[complex(0.60,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

# Análise das respostas

minimos9 = np.zeros((1,2000), dtype=np.complex_) # Verificar se o mínimo será zero
constantes9 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1
valoresOnda9 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar as componentes de cada onda
valoresFuncaoObjetivo9 = np.zeros((1,2000), dtype=np.complex_) # Conferir se o mínimo está realmente ajustado

# Hamiltoniano

hamiltoniano = np.zeros((2,2), dtype=np.complex_) 
hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]

iteracao = 0
tempoAnalise = 2000

while iteracao < tempoAnalise:
    
    print(f"Iteração: {iteracao+1}")
    
    # Variação do Psi
    valoresOnda9[0][iteracao] = ondaOrigem[0][0]
    valoresOnda9[1][iteracao] = ondaOrigem[1][0]
    
    # Obtenção das constantes que multiplicam Psi
    constantes9[0][iteracao] = ondaOrigem[0][0] / np.exp(-1*complex(0,1)*(1/2*np.pi)*iteracao/100)
    constantes9[1][iteracao] = ondaOrigem[1][0] / np.exp(-3*complex(0,1)*(1/2*np.pi)*iteracao/100)
        
    ## Minimização
    
    resultado = optimize.minimize(funcaoObjetivo, x, args=(iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise), method='CG')

    print(f"Mínimo: {resultado.x}")
    
    # Função objetivo
    
    minimos9[0][iteracao] = resultado.x[0]
    
    valoresFuncaoObjetivo9[0][iteracao] = funcaoObjetivo(resultado.x, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    
    print(f" Conferindo: {valoresFuncaoObjetivo9[0][iteracao]}")
    
    ## Hamiltoniano corntrolado
    
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, resultado.x[0]],[resultado.x[0], 0]]
    hamiltoniano = somar(hamiltoniano, controle)
    
    ## Adaptação da onda ao controle
    
    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)
    
    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]
    
    # Reset
    hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]
    
    x = []
    for l in range(0, horizonte):
        x.append(resultado.x[l])
    
    iteracao += 1

Iteração: 1
Mínimo: [-2.80889716  0.82292187  0.92015374  0.85383667  0.8833271   0.89296882
  0.89073642  0.86229572 -0.05952453]
 Conferindo: (0.17760458672025614+0j)
Iteração: 2
Mínimo: [ 0.44272718  0.95453672  0.79126372  0.86246805  0.87338727  0.86976032
  0.88425909  0.88345057 -0.05952453]
 Conferindo: (0.1762520531344309+0j)
Iteração: 3
Mínimo: [ 0.44302605  0.94590875  0.80953375  0.869935    0.86746561  0.85432539
  0.86263824  0.8667518  -0.05952453]
 Conferindo: (0.17372184392093662+0j)
Iteração: 4
Mínimo: [ 0.44183721  0.94321537  0.80846677  0.86820454  0.86544538  0.85243205
  0.86086676  0.86553277 -0.05952453]
 Conferindo: (0.17120873791286534+0j)
Iteração: 5
Mínimo: [ 0.4596357   0.939724    0.81362622  0.86215813  0.8527256   0.83773525
  0.8450649   0.8537234  -0.05952453]
 Conferindo: (0.1687148030677968+0j)
Iteração: 6
Mínimo: [ 0.45286472  0.93183029  0.80743583  0.85633746  0.84775191  0.83388139
  0.84218976  0.85201492 -0.05952453]
 Conferindo: (0.1662431013

Mínimo: [ 0.33476251  0.65360835  0.59572701  0.6061087   0.59333861  0.5842581
  0.58607276  0.62954317 -0.05952453]
 Conferindo: (0.08743622784986588+0j)
Iteração: 51
Mínimo: [ 0.32705644  0.64548104  0.58873552  0.59999463  0.58835714  0.58036307
  0.58302695  0.62727203 -0.05952453]
 Conferindo: (0.08619658326656807+0j)
Iteração: 52
Mínimo: [ 0.32062648  0.6386162   0.58288427  0.59482237  0.58407392  0.57691619
  0.58020908  0.62506871 -0.05952453]
 Conferindo: (0.08498159494955086+0j)
Iteração: 53
Mínimo: [ 0.31533377  0.63278363  0.57789903  0.59030872  0.58022954  0.57370492
  0.57746826  0.62285153 -0.05952453]
 Conferindo: (0.0837882062571157+0j)
Iteração: 54
Mínimo: [ 0.31102869  0.6277689   0.5735422   0.58622103  0.57661884  0.57056644
  0.57469077  0.6205604  -0.05952453]
 Conferindo: (0.08261411363243114+0j)
Iteração: 55
Mínimo: [ 0.30749754  0.62330968  0.56955275  0.5823152   0.57303211  0.56733377
  0.57175489  0.61812065 -0.05952453]
 Conferindo: (0.08145765537667551

Mínimo: [ 0.22548737  0.43862356  0.40176234  0.40253074  0.3945189   0.38835718
  0.38435732  0.42822282 -0.05952453]
 Conferindo: (0.04541984908557918+0j)
Iteração: 100
Mínimo: [ 0.21972043  0.43260173  0.39656799  0.39810999  0.39097077  0.38561515
  0.38224033  0.42649534 -0.05952453]
 Conferindo: (0.04486540472060586+0j)
Iteração: 101
Mínimo: [ 0.21488846  0.42752509  0.39224078  0.39441557  0.38797024  0.38323505
  0.38031859  0.42483633 -0.05952453]
 Conferindo: (0.04432294950166286+0j)
Iteração: 102
Mínimo: [ 0.21093494  0.42326232  0.38861007  0.39126485  0.38534721  0.38107578
  0.37849157  0.42319256 -0.05952453]
 Conferindo: (0.04379089188592991+0j)
Iteração: 103
Mínimo: [ 0.20771515  0.41962135  0.3854661   0.3884526   0.38292166  0.37899363
  0.37665862  0.42150864 -0.05952453]
 Conferindo: (0.04326801781618658+0j)
Iteração: 104
Mínimo: [ 0.20510117  0.41643564  0.38263775  0.38581862  0.38055342  0.37687758
  0.3747402   0.41973564 -0.05952453]
 Conferindo: (0.0427534467

Mínimo: [ 0.14097347  0.29231896  0.26677769  0.26827393  0.26539003  0.26341071
  0.26045462  0.30030923 -0.05952453]
 Conferindo: (0.027122514868446634+0j)
Iteração: 148
Mínimo: [ 0.14097347  0.29231896  0.26677769  0.26827393  0.26539003  0.26341071
  0.26045462  0.30030923 -0.05952453]
 Conferindo: (0.026876569224053983+0j)
Iteração: 149
Mínimo: [ 0.1385155   0.2888814   0.2632484   0.26467773  0.26190766  0.26020158
  0.25767983  0.29808441 -0.05952453]
 Conferindo: (0.026633823691248035+0j)
Iteração: 150
Mínimo: [ 0.15249349  0.29154744  0.2625362   0.25998474  0.25401194  0.24965031
  0.24511217  0.28387011 -0.05952453]
 Conferindo: (0.026394432127521593+0j)
Iteração: 151
Mínimo: [ 0.14700212  0.28591794  0.25770997  0.25599548  0.25089398  0.24730791
  0.2433674   0.28244608 -0.05952453]
 Conferindo: (0.026158677214931522+0j)
Iteração: 152
Mínimo: [ 0.14240627  0.2812874   0.25387498  0.25289967  0.24850928  0.24550655
  0.24196365  0.28117592 -0.05952453]
 Conferindo: (0.02592

Mínimo: [ 0.10018462  0.198552    0.17400648  0.17257961  0.16919639  0.16690118
  0.16383356  0.20110259 -0.05952453]
 Conferindo: (0.019029035636535747+0j)
Iteração: 196
Mínimo: [ 0.09682412  0.19526292  0.17147458  0.17068674  0.16783576  0.16590096
  0.16298747  0.20013781 -0.05952453]
 Conferindo: (0.0189223972489388+0j)
Iteração: 197
Mínimo: [ 0.09414283  0.19263792  0.16953901  0.16928431  0.16684264  0.16514737
  0.1622816   0.19922575 -0.05952453]
 Conferindo: (0.018818257353494475+0j)
Iteração: 198
Mínimo: [ 0.09204769  0.19053959  0.16803845  0.16820698  0.16606668  0.16451753
  0.16163053  0.1983236  -0.05952453]
 Conferindo: (0.018716182594430288+0j)
Iteração: 199
Mínimo: [ 0.09204769  0.19053959  0.16803845  0.16820698  0.16606668  0.16451753
  0.16163053  0.1983236  -0.05952453]
 Conferindo: (0.018615884524430294+0j)
Iteração: 200
Mínimo: [ 0.08897096  0.18739377  0.16542694  0.16601456  0.16424349  0.16299076
  0.16031902  0.19708582 -0.05952453]
 Conferindo: (0.0185170

...

Mínimo: [-8.65118199e-04  9.58923519e-04 -2.48263255e-03 -3.59266087e-04
 -1.83325980e-04  8.94066327e-05 -3.57988494e-03  3.30508171e-03
 -5.95245331e-02]
 Conferindo: (0.01278946562475996+0j)
Iteração: 1792
Mínimo: [ 0.00093861  0.00257575 -0.00101485  0.00088066  0.00082309  0.00086288
 -0.0030287   0.00351964 -0.05952453]
 Conferindo: (0.012789440246797677+0j)
Iteração: 1793
Mínimo: [ 0.00093861  0.00257575 -0.00101485  0.00088066  0.00082309  0.00086288
 -0.0030287   0.00351964 -0.05952453]
 Conferindo: (0.012789439411246087+0j)
Iteração: 1794
Mínimo: [ 0.00093861  0.00257575 -0.00101485  0.00088066  0.00082309  0.00086288
 -0.0030287   0.00351964 -0.05952453]
 Conferindo: (0.012789440339546607+0j)
Iteração: 1795
Mínimo: [ 0.00093861  0.00257575 -0.00101485  0.00088066  0.00082309  0.00086288
 -0.0030287   0.00351964 -0.05952453]
 Conferindo: (0.012789443032012963+0j)
Iteração: 1796
Mínimo: [ 0.00093861  0.00257575 -0.00101485  0.00088066  0.00082309  0.00086288
 -0.0030287   0.00

Mínimo: [-1.06500270e-03  6.12658808e-04 -2.59430922e-03 -5.57016601e-04
 -3.85384241e-04 -2.44449762e-05 -3.47508298e-03  2.99808439e-03
 -5.95245331e-02]
 Conferindo: (0.012789417843553821+0j)
Iteração: 1836
Mínimo: [-1.06500270e-03  6.12658808e-04 -2.59430922e-03 -5.57016601e-04
 -3.85384241e-04 -2.44449762e-05 -3.47508298e-03  2.99808439e-03
 -5.95245331e-02]
 Conferindo: (0.012789423459705131+0j)
Iteração: 1837
Mínimo: [-1.06500270e-03  6.12658808e-04 -2.59430922e-03 -5.57016601e-04
 -3.85384241e-04 -2.44449762e-05 -3.47508298e-03  2.99808439e-03
 -5.95245331e-02]
 Conferindo: (0.012789430933780604+0j)
Iteração: 1838
Mínimo: [ 0.00066446  0.00218844 -0.0011464   0.00068032  0.00062885  0.00075943
 -0.00291539  0.00322266 -0.05952453]
 Conferindo: (0.012789407798315907+0j)
Iteração: 1839
Mínimo: [ 0.00066446  0.00218844 -0.0011464   0.00068032  0.00062885  0.00075943
 -0.00291539  0.00322266 -0.05952453]
 Conferindo: (0.012789407119656709+0j)
Iteração: 1840
Mínimo: [ 0.00066446  0.

Mínimo: [-0.0009642   0.00063353 -0.00233908 -0.00040125 -0.00027153  0.00011034
 -0.00320162  0.00280623 -0.05952453]
 Conferindo: (0.012789382661030115+0j)
Iteração: 1883
Mínimo: [-0.0009642   0.00063353 -0.00233908 -0.00040125 -0.00027153  0.00011034
 -0.00320162  0.00280623 -0.05952453]
 Conferindo: (0.012789387382469736+0j)
Iteração: 1884
Mínimo: [-0.0009642   0.00063353 -0.00233908 -0.00040125 -0.00027153  0.00011034
 -0.00320162  0.00280623 -0.05952453]
 Conferindo: (0.012789393646062188+0j)
Iteração: 1885
Mínimo: [-0.0009642   0.00063353 -0.00233908 -0.00040125 -0.00027153  0.00011034
 -0.00320162  0.00280623 -0.05952453]
 Conferindo: (0.0127894014469527+0j)
Iteração: 1886
Mínimo: [ 0.00087305  0.00229204 -0.00083415  0.00087485  0.00076712  0.00090736
 -0.00263625  0.00303727 -0.05952453]
 Conferindo: (0.012789375391875905+0j)
Iteração: 1887
Mínimo: [ 0.00087305  0.00229204 -0.00083415  0.00087485  0.00076712  0.00090736
 -0.00263625  0.00303727 -0.05952453]
 Conferindo: (0.01

 Conferindo: (0.012789350782617568+0j)
Iteração: 1930
Mínimo: [-0.00094485  0.00055119 -0.00221801 -0.00037888 -0.00028139  0.00013599
 -0.00301297  0.00259526 -0.05952453]
 Conferindo: (0.012789355391624518+0j)
Iteração: 1931
Mínimo: [-0.00094485  0.00055119 -0.00221801 -0.00037888 -0.00028139  0.00013599
 -0.00301297  0.00259526 -0.05952453]
 Conferindo: (0.012789361493268352+0j)
Iteração: 1932
Mínimo: [-0.00094485  0.00055119 -0.00221801 -0.00037888 -0.00028139  0.00013599
 -0.00301297  0.00259526 -0.05952453]
 Conferindo: (0.012789369082797661+0j)
Iteração: 1933
Mínimo: [ 0.00087687  0.0021955  -0.00072829  0.0008841   0.00074645  0.00092375
 -0.00245504  0.00282641 -0.05952453]
 Conferindo: (0.0127893435459564+0j)
Iteração: 1934
Mínimo: [ 0.00087687  0.0021955  -0.00072829  0.0008841   0.00074645  0.00092375
 -0.00245504  0.00282641 -0.05952453]
 Conferindo: (0.012789342590392966+0j)
Iteração: 1935
Mínimo: [ 0.00087687  0.0021955  -0.00072829  0.0008841   0.00074645  0.00092375
 -

 Conferindo: (0.012789316106476375+0j)
Iteração: 1976
Mínimo: [-0.00093887  0.00046263 -0.00211729 -0.00037229 -0.00030247  0.00014758
 -0.00284116  0.00239825 -0.05952453]
 Conferindo: (0.012789319223262145+0j)
Iteração: 1977
Mínimo: [-0.00093887  0.00046263 -0.00211729 -0.00037229 -0.00030247  0.00014758
 -0.00284116  0.00239825 -0.05952453]
 Conferindo: (0.01278932382860316+0j)
Iteração: 1978
Mínimo: [-0.00093887  0.00046263 -0.00211729 -0.00037229 -0.00030247  0.00014758
 -0.00284116  0.00239825 -0.05952453]
 Conferindo: (0.012789329919197364+0j)
Iteração: 1979
Mínimo: [-0.00093887  0.00046263 -0.00211729 -0.00037229 -0.00030247  0.00014758
 -0.00284116  0.00239825 -0.05952453]
 Conferindo: (0.012789337490276801+0j)
Iteração: 1980
Mínimo: [ 0.00088126  0.00210711 -0.00062967  0.00088925  0.00072451  0.00093387
 -0.00228544  0.0026319  -0.05952453]
 Conferindo: (0.012789311909090969+0j)
Iteração: 1981
Mínimo: [ 0.00088126  0.00210711 -0.00062967  0.00088925  0.00072451  0.00093387
 

In [12]:
## Constantes

horizonte = 10
fatorRungeKutta = 0.01

## Chute inicial

x = []
x.append(random.uniform(-1, 1))
for l in range(1, horizonte):
    x.append(x[0])

# Função de origem e destino

ondaOrigem = [[complex(0.80,0.0)],[complex(0.60,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]

# Análise das respostas

minimos10 = np.zeros((1,2000), dtype=np.complex_) # Verificar se o mínimo será zero
constantes10 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar se as integrais estão resultando em 1
valoresOnda10 = np.zeros((2, 2000), dtype=np.complex_) # Para verificar as componentes de cada onda
valoresFuncaoObjetivo10 = np.zeros((1,2000), dtype=np.complex_) # Conferir se o mínimo está realmente ajustado

# Hamiltoniano

hamiltoniano = np.zeros((2,2), dtype=np.complex_) 
hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]

iteracao = 0
tempoAnalise = 2000

while iteracao < tempoAnalise:
    
    print(f"Iteração: {iteracao+1}")
    
    # Variação do Psi
    valoresOnda10[0][iteracao] = ondaOrigem[0][0]
    valoresOnda10[1][iteracao] = ondaOrigem[1][0]
    
    # Obtenção das constantes que multiplicam Psi
    constantes10[0][iteracao] = ondaOrigem[0][0] / np.exp(-1*complex(0,1)*(1/2*np.pi)*iteracao/100)
    constantes10[1][iteracao] = ondaOrigem[1][0] / np.exp(-3*complex(0,1)*(1/2*np.pi)*iteracao/100)
        
    ## Minimização
    
    resultado = optimize.minimize(funcaoObjetivo, x, args=(iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise), method='CG')

    print(f"Mínimo: {resultado.x}")
    
    # Função objetivo
    
    minimos10[0][iteracao] = resultado.x[0]
    
    valoresFuncaoObjetivo10[0][iteracao] = funcaoObjetivo(resultado.x, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonte, fatorRungeKutta, tempoAnalise)
    
    print(f" Conferindo: {valoresFuncaoObjetivo10[0][iteracao]}")
    
    ## Hamiltoniano corntrolado
    
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, resultado.x[0]],[resultado.x[0], 0]]
    hamiltoniano = somar(hamiltoniano, controle)
    
    ## Adaptação da onda ao controle
    
    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)
    
    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]
    
    # Reset
    hamiltoniano = [[(1/2*np.pi), 0], [0, (3/2*np.pi)]]
    
    x = []
    for l in range(0, horizonte):
        x.append(resultado.x[l])
    
    iteracao += 1

Iteração: 1
Mínimo: [-3.22741052  0.79436165  0.97298706  0.85158169  0.85439485  0.88022778
  0.87833889  0.87078718  0.85606839  0.54779504]
 Conferindo: (0.19633281631991972+0j)
Iteração: 2
Mínimo: [0.43018525 0.92682074 0.80845858 0.90110898 0.88897299 0.84812704
 0.84167853 0.85606562 0.86131749 0.54779504]
 Conferindo: (0.19465673937867872+0j)
Iteração: 3
Mínimo: [0.45457833 0.93412904 0.81647947 0.89178459 0.87475047 0.83872816
 0.83500457 0.84891536 0.85567205 0.54779504]
 Conferindo: (0.19150489053316816+0j)
Iteração: 4
Mínimo: [0.46475222 0.92730919 0.81541835 0.87951681 0.86255129 0.8334318
 0.8322562  0.84439091 0.85097886 0.54779504]
 Conferindo: (0.18836863266102946+0j)
Iteração: 5
Mínimo: [0.45816243 0.91901331 0.80852266 0.8720928  0.85594801 0.82840348
 0.82836202 0.84129075 0.8490384  0.54779504]
 Conferindo: (0.18527057286450654+0j)
Iteração: 6
Mínimo: [0.46671267 0.91176012 0.80675285 0.86075818 0.84364099 0.81972324
 0.81989692 0.83093823 0.84026922 0.54779504]
 Co

Mínimo: [0.31366462 0.62012261 0.57435886 0.58444458 0.57473657 0.56949238
 0.56643239 0.56772559 0.60951896 0.54779504]
 Conferindo: (0.08991706975934398+0j)
Iteração: 50
Mínimo: [0.31888116 0.61655617 0.57138764 0.57877494 0.567771   0.56104827
 0.55599393 0.55497624 0.59557268 0.54779504]
 Conferindo: (0.08848687388183565+0j)
Iteração: 51
Mínimo: [0.31416973 0.61135829 0.56683968 0.57466546 0.56430875 0.5582198
 0.553696   0.55305156 0.59392848 0.54779504]
 Conferindo: (0.08707981149946348+0j)
Iteração: 52
Mínimo: [0.30855371 0.60523254 0.56132942 0.56956677 0.55984953 0.55443124
 0.55053908 0.55045234 0.5918921  0.54779504]
 Conferindo: (0.08569952332787965+0j)
Iteração: 53
Mínimo: [0.30268754 0.59844016 0.55512975 0.56363174 0.55447939 0.54967711
 0.54638718 0.54687186 0.58899306 0.54779504]
 Conferindo: (0.08434438966092579+0j)
Iteração: 54
Mínimo: [0.30811095 0.59510637 0.5521857  0.55809785 0.54762597 0.54130716
 0.53609482 0.53436517 0.57514519 0.54779504]
 Conferindo: (0.0830

Mínimo: [0.20750701 0.40698311 0.37553988 0.37485258 0.36863604 0.36532095
 0.36187059 0.3596263  0.40218086 0.54779504]
 Conferindo: (0.04352742810501396+0j)
Iteração: 98
Mínimo: [0.2022434  0.4012185  0.37048578 0.37040454 0.36483762 0.36208689
 0.35910408 0.35719614 0.39996088 0.54779504]
 Conferindo: (0.04292963271499195+0j)
Iteração: 99
Mínimo: [0.20037141 0.39911863 0.36858486 0.36867014 0.3632987  0.36073238
 0.35792377 0.35617202 0.39907834 0.54779504]
 Conferindo: (0.042343364116608335+0j)
Iteração: 100
Mínimo: [0.19622489 0.39416738 0.36400861 0.36436755 0.35935116 0.35712343
 0.35465947 0.35325186 0.39652852 0.54779504]
 Conferindo: (0.0417671710283257+0j)
Iteração: 101
Mínimo: [0.19392517 0.39120669 0.36116175 0.36158798 0.35671393 0.35464664
 0.35238067 0.35120801 0.39477912 0.54779504]
 Conferindo: (0.04120091963151839+0j)
Iteração: 102
Mínimo: [0.19221303 0.38868305 0.35861387 0.35898547 0.35415021 0.35216287
 0.35004033 0.34908108 0.39296386 0.54779504]
 Conferindo: (0.

Mínimo: [0.12857362 0.26214921 0.23756668 0.23783832 0.23531351 0.23394309
 0.23175421 0.22892278 0.2686707  0.54779504]
 Conferindo: (0.02431755604957438+0j)
Iteração: 146
Mínimo: [0.12720703 0.26029912 0.23584399 0.23616917 0.23371113 0.23240909
 0.230309   0.22757215 0.26737774 0.54779504]
 Conferindo: (0.024073455868419778+0j)
Iteração: 147
Mínimo: [0.12604672 0.25855204 0.23414075 0.23445192 0.23201045 0.2307471
 0.22872715 0.22610004 0.26600471 0.54779504]
 Conferindo: (0.023833230673911892+0j)
Iteração: 148
Mínimo: [0.12504459 0.25684202 0.23239775 0.23263407 0.23016595 0.22891547
 0.22696982 0.2244678  0.26450937 0.54779504]
 Conferindo: (0.023596784028502744+0j)
Iteração: 149
Mínimo: [0.12423235 0.25520119 0.2306529  0.23075905 0.22822358 0.22695948
 0.22507631 0.22270406 0.26290568 0.54779504]
 Conferindo: (0.023364065797970957+0j)
Iteração: 150
Mínimo: [0.1338971  0.25675802 0.23037809 0.2281549  0.2235351  0.22028683
 0.21672064 0.21291056 0.25166731 0.54779504]
 Conferindo

Mínimo: [0.08164577 0.17162377 0.15023973 0.15035574 0.14867663 0.14777047
 0.14642105 0.1440362  0.18096055 0.54779504]
 Conferindo: (0.016508126976721375+0j)
Iteração: 194
Mínimo: [0.08042376 0.17024654 0.14900846 0.14923713 0.14766636 0.14685983
 0.14560709 0.1433118  0.18029398 0.54779504]
 Conferindo: (0.0164099708246165+0j)
Iteração: 195
Mínimo: [0.07855757 0.16801499 0.14696498 0.14732622 0.1458903  0.14521515
 0.14410373 0.1419556  0.17904805 0.54779504]
 Conferindo: (0.01631337768033801+0j)
Iteração: 196
Mínimo: [0.07855757 0.16801499 0.14696498 0.14732622 0.1458903  0.14521515
 0.14410373 0.1419556  0.17904805 0.54779504]
 Conferindo: (0.016218293565685674+0j)
Iteração: 197
Mínimo: [0.07734563 0.16624721 0.14521237 0.14555968 0.14414033 0.14351091
 0.14249157 0.14048144 0.17771935 0.54779504]
 Conferindo: (0.016124492096486063+0j)
Iteração: 198
Mínimo: [0.07734563 0.16624721 0.14521237 0.14555968 0.14414033 0.14351091
 0.14249157 0.14048144 0.17771935 0.54779504]
 Conferindo:

...

Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262268588125294+0j)
Iteração: 1810
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262270802360847+0j)
Iteração: 1811
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.01126227300532437+0j)
Iteração: 1812
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262275196267586+0j)
Iteração: 1813
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262277374453329+0j)
Iteração: 1814
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 

Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262351341149768+0j)
Iteração: 1857
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.0112623525086562+0j)
Iteração: 1858
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262353650982021+0j)
Iteração: 1859
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262354768418319+0j)
Iteração: 1860
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262355861279596+0j)
Iteração: 1861
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -

Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262387402328556+0j)
Iteração: 1904
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.01126238801388535+0j)
Iteração: 1905
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262388632084855+0j)
Iteração: 1906
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262389257745902+0j)
Iteração: 1907
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262389891682578+0j)
Iteração: 1908
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 

Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262434270504676+0j)
Iteração: 1951
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262435843121482+0j)
Iteração: 1952
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262437442721849+0j)
Iteração: 1953
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262439069168509+0j)
Iteração: 1954
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262440722295843+0j)
Iteração: 1955
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349


Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.01126253088365341+0j)
Iteração: 1998
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262533209280114+0j)
Iteração: 1999
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.01126253553333876+0j)
Iteração: 2000
Mínimo: [-0.00099033  0.01337781 -0.00456975 -0.00470062 -0.00534874 -0.0053349
 -0.00489182 -0.00473587  0.02966246  0.54779504]
 Conferindo: (0.011262537854965983+0j)
